# Flask を使った簡易Webアプリ

まずは必要なライブラリをインストール

In [1]:
!pip install -r requirements.txt

  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached flask_cors-5.0.1-py3-none-any.whl.metadata (961 bytes)
  Using cached SpeechRecognition-3.14.1-py3-none-any.whl.metadata (31 kB)
  Using cached openai-1.65.2-py3-none-any.whl.metadata (27 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached Flask_SocketIO-5.5.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


## その１　とりあえず音声ファイルをアップロードできるようにする．

In [42]:
%%writefile static/index.html

<!DOCTYPE html>
<html lang="ja">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Voice Chat App</title>
  </head>
  <body>
    <h1>Voice Chat App</h1>
    <button id="start">開始</button>
    <button id="stop" disabled>停止</button>
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
      document.addEventListener("DOMContentLoaded", () => {
        const startButton = document.getElementById("start");
        const stopButton = document.getElementById("stop");
        const transcriptionElement = document.getElementById("transcription");
        const aiResponseElement = document.getElementById("aiResponse");
        let mediaRecorder;
        let audioChunks = [];

        startButton.addEventListener("click", async () => {
          const stream = await navigator.mediaDevices.getUserMedia({
            audio: true,
          });
          mediaRecorder = new MediaRecorder(stream);

          mediaRecorder.ondataavailable = (event) => {
            audioChunks.push(event.data);
          };

          mediaRecorder.onstop = async () => {
            const audioBlob = new Blob(audioChunks, { type: "audio/webm" });
            audioChunks = [];

            const formData = new FormData();
            formData.append("audio", audioBlob, "recording.webm");

            fetch("/upload", {
              method: "POST",
              body: formData,
            })
              .then((response) => response.json())
              .then((data) => {
                transcriptionElement.textContent =
                  data.text || "認識できませんでした。";
                aiResponseElement.textContent =
                  data.ai_response || "AIの応答なし。";
              })
              .catch((error) => {
                console.error("Upload failed:", error);
                transcriptionElement.textContent = "エラーが発生しました。";
                aiResponseElement.textContent = "";
              });
          };

          mediaRecorder.start();
          startButton.disabled = true;
          stopButton.disabled = false;
        });

        stopButton.addEventListener("click", () => {
          mediaRecorder.stop();
          startButton.disabled = false;
          stopButton.disabled = true;
        });
      });
    </script>
  </body>
</html>


Overwriting static/index.html


In [ ]:
%%writefile app01.py

from flask import Flask, request, jsonify, send_from_directory
import os

app = Flask(__name__, static_folder="static")  

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "audio" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["audio"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)
    
    text = "test"
    ai_response = "test"
    return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Overwriting app01.py


ふむ．とりあえず音声ファイル.webmはアップロードできるようになった．
ただspeech_recgnitionではwebmは受け入れないので，wavファイルに変える必要がある

## その２　アップロードした音声ファイルをWavファイルに変換

In [ ]:
%%writefile app02.py

from flask import Flask, request, jsonify, render_template
import os
import subprocess

app = Flask(__name__)

@app.route("/")
def index():
    return render_template("index.html")  # フロントエンドのHTMLを表示

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "audio" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["audio"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)
    convert_webm_to_wav(audio_path, "uploads/output.wav")
    
    text = "test"
    ai_response = "test"
    return jsonify({"text": text, "ai_response": ai_response})


def convert_webm_to_wav(input_path, output_path):
    command = [
        "ffmpeg",
        "-i", input_path,  # 入力ファイル
        "-ar", "16000",  # サンプリングレート 16kHz
        "-ac", "1",  # モノラル変換
        "-preset", "ultrafast",  # 速度最優先
        output_path
    ]
    subprocess.run(command, check=True)

# 使い方

if __name__ == "__main__":
    app.run(debug=True)


Writing app02.py


変換に少々時間取られるな．．．

## その３　アップロードを直接Wavファイルにする．

変換に少々時間がかかるのが気になるので，アップロードの段階で直接Wavファイルをアップできないか探ってみたら，Record.jsなるものがあるようだ．
https://github.com/mattdiamond/Recorderjs
Recorder.jsをダウンロードして
これをhtmlに組み込んでみる．

けど，最初やってみたら，recorder.jsでエラーでた．
CDNがあるようなので，そちらでやったらうまく行った．

In [52]:
%%writefile static/index.html

<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>
    <!-- <script src="recorder.js"></script> --> 
    
</head>
<body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <!-- <audio id="audioPlayback" controls></audio> -->
    <!-- <button id="uploadAudio" disabled>アップロード</button> -->
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
        let audioContext;
        let recorder;
        let audioBlob;
                

        document.getElementById("startRecording").addEventListener("click", async () => {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
        });

        document.getElementById("stopRecording").addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
                audioBlob = blob;

                if (!audioBlob) {
                    console.error("No audio to upload");    
                    return;
                }

                const formData = new FormData();
                formData.append("file", audioBlob, "recorded_audio.wav");

                fetch("/upload", {
                    method: "POST",
                    body: formData,
                })
                .then((response) => response.json())
                .then((data) => {
                    document.getElementById("transcription").textContent =
                    data.text || "認識できませんでした。";
                    document.getElementById("aiResponse").textContent =
                    data.ai_response || "AIの応答なし。";
                })
                .catch((error) => {
                    console.error("Upload failed:");
                    document.getElementById("transcription").textContent = "エラーが発生しました。";
                    document.getElementById("aiResponse").textContent = "";
                });
            });

            document.getElementById("startRecording").disabled = false;
            document.getElementById("stopRecording").disabled = true;


        });
    </script>
</body>
</html>


Overwriting static/index.html


app01.pyで実行．
CORSの問題でアップロードで弾かれているようだ．．．

flask_corsを使って，サーバの側でCORS問題を無視するように設定する．

In [ ]:
!pip install flask_cors

In [40]:
%%writefile app03.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os

app = Flask(__name__, static_folder="static")  
CORS(app)

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)
    
    text = "test"
    ai_response = "test"
    return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Overwriting app03.py


よし，とりあえず問題は解決した．
ハマった理由は，fetchのインタフェースをフォルダ名と勘違いしていたこと．つまり，app.routeでは/uploadとしているのに，javascriptの方で/uploadsとしていた．これにより当然ながらインタフェースがないわけで４０４エラーが返されれるということになっていた．分れば馬鹿馬鹿しい勘違いやった😂

あと，デバッグ環境ではルートディレクトリがprojectになるというところもハマった😂

## その４　Speech Recognitionにかける
よし，ここからはpythonの側の処理に集中

In [54]:
%%writefile app04.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import speech_recognition as sr

app = Flask(__name__, static_folder="static")  
CORS(app)

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)
            
        ai_response = "test"
        return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Overwriting app04.py


ふむ．これでとりあえず，音声認識結果を返せるようになった．

## その５　 openai の　Chat＿Compelationを使う

In [55]:
%%writefile app05.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

app = Flask(__name__, static_folder="static")  
CORS(app)

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)

        # AIの応答
        client = OpenAI()
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": text},
            ]
        )
        ai_response = completion.choices[0].message.content
        return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Writing app05.py


ふむ．とりあえずは単発会話はできるようになった．
現時点の違和感・修正したい点は
現状だと，入力テキストとレスポンスが同時に帰ってきてしまう．
どうにか，その部分をいじれないか．レスポンスをまたつに先に入力テキストを返して，画面に表示させておいて，レスポンスが帰ってきたら，改めてそれを返すという感じ．

## その6　入力テキストとレスポンスを分けて表記できるようにする
やるとしたら，postを2回に分ける形かな？
Copilotに聞いてみたらWeb Socketを使えばできるとな．．．
とりあえずやってみるか．

In [ ]:
!pip install flask-socketio

In [5]:
%%writefile app06.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading

load_dotenv()

app = Flask(__name__, static_folder="static")  
CORS(app)
socketio = SocketIO(app)

@app.route("/")
def index():
    return send_from_directory("static", "index06.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

def get_ai_response(text):
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": text},
        ]
    )
    ai_response = completion.choices[0].message.content
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response})

if __name__ == "__main__":
    socketio.run(app, debug=True)

Overwriting app06.py


In [7]:
%%writefile static/index06.html

<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    
</head>
<body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <!-- <audio id="audioPlayback" controls></audio> -->
    <!-- <button id="uploadAudio" disabled>アップロード</button> -->
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
        let audioContext;
        let recorder;
        let audioBlob;

        document.addEventListener("DOMContentLoaded", () => {
            const socket = io();

            socket.on('ai_response', (data) => {
                document.getElementById("aiResponse").textContent = data.ai_response;
            });
        });        

        document.getElementById("startRecording").addEventListener("click", async () => {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
        });

        document.getElementById("stopRecording").addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
                audioBlob = blob;

                if (!audioBlob) {
                    console.error("No audio to upload");    
                    return;
                }

                const formData = new FormData();
                formData.append("file", audioBlob, "recorded_audio.wav");

                fetch("/upload", {
                    method: "POST",
                    body: formData,
                })
                .then((response) => response.json())
                .then((data) => {
                    document.getElementById("transcription").textContent =
                    data.text || "認識できませんでした。";
                })
                .catch((error) => {
                    console.error("Upload failed:");
                    document.getElementById("transcription").textContent = "エラーが発生しました。";
                    document.getElementById("aiResponse").textContent = "";
                });
            });

            document.getElementById("startRecording").disabled = false;
            document.getElementById("stopRecording").disabled = true;


        });
    </script>
</body>
</html>


Overwriting static/index06.html


ふむ．入力とレスポンスを別々に表記できるようになった．

## その７　継続的な会話を出来るようにする．
とりあえず，フロントエンドでログを記載する部分については別に考えて，まずはパイソンで動かす

In [8]:
%%writefile app07.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading

load_dotenv()

app = Flask(__name__, static_folder="static")  
CORS(app)
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

@app.route("/")
def index():
    return send_from_directory("static", "index06.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

def get_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response})

if __name__ == "__main__":
    socketio.run(app, debug=True)

Writing app07.py


ふむ．継続性のある会話もできるようになった．
じゃあ次は，TTSを組み込みたいね．

調べたらVoice VoxであればAPIが使えるとのこと．ただ，このAPIはあくまでローカルサーバで動くものになっている．
ローカルでどれくらい動作時間かかるんやろ？？
とりあえず試すか・・・．

## その8　音声合成機能を使ってみる
VoiceVox GUIを立ち上げておく．これによりローカルにVoiceVoxが立ち上がりAPIが使える．
そのまま関数を作ってくれている人がいたので拝借

https://zenn.dev/zenn24yykiitos/articles/fff3c954ddf42c

In [9]:
!pip install requests


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%%writefile app08.py
from flask import Flask, request, jsonify, send_from_directory, send_file
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading
import requests
import json
import time

#　環境変数の読み込み
load_dotenv()

# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    return send_from_directory("static", "index08.html")

# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

# 音声ファイルを提供するエンドポイント
@app.route("/audio/<filename>")
def get_audio(filename):
    return send_file(os.path.join("uploads",filename))


#--------------------------------------------------

# AIの応答を取得する関数 
def get_ai_response(text):
    
    # 現在の時刻取得
    start = time.time()

    # OpenAIのAPIを呼び出してAIの応答を取得
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})

    # 処理時間の計算
    ai_time = time.time() - start
    print(f"処理時間: {ai_time} [sec]") 

    # 音声合成
    filename = synthesize_voice(ai_response)

    # 処理時間の計算
    voice_time = time.time() - start - ai_time
    print(f"音声合成時間: {voice_time} [sec]")
    
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response, 'audio': filename})



# 音声合成を行なう関数
def synthesize_voice(text, speaker=1, filename="uploads/output.wav"):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'http://localhost:50021/audio_query', params=query_payload)

    if query_response.status_code != 200:
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'http://localhost:50021/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        # 音声ファイルとして保存
        with open(filename, 'wb') as f:
            f.write(synthesis_response.content)
        print(f"音声が {filename} に保存されました。")
        return "output.wav"
    else:
        print(f"Error in synthesis: {synthesis_response.text}")
        return None


if __name__ == "__main__":
    socketio.run(app, debug=True)

Writing app08.py


In [ ]:
%%writefile static/index08.html

<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    
</head>
<body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <!-- <audio id="audioPlayback" controls></audio> -->
    <!-- <button id="uploadAudio" disabled>アップロード</button> -->
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
        let audioContext;
        let recorder;
        let audioBlob;

        document.addEventListener("DOMContentLoaded", () => {
            const socket = io();

            socket.on('ai_response', (data) => {
                document.getElementById("aiResponse").textContent = data.ai_response;
                
                // 音声ファイルを自動再生する処理
                if (data.audio) {
                    const audio = new Audio(`/audio/${data.audio}`);
                    audio.play();
                }    
            });
        });        

        document.getElementById("startRecording").addEventListener("click", async () => {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
        });

        document.getElementById("stopRecording").addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
                audioBlob = blob;

                if (!audioBlob) {
                    console.error("No audio to upload");    
                    return;
                }

                const formData = new FormData();
                formData.append("file", audioBlob, "recorded_audio.wav");

                fetch("/upload", {
                    method: "POST",
                    body: formData,
                })
                .then((response) => response.json())
                .then((data) => {
                    document.getElementById("transcription").textContent =
                    data.text || "認識できませんでした。";
                })
                .catch((error) => {
                    console.error("Upload failed:");
                    document.getElementById("transcription").textContent = "エラーが発生しました。";
                    document.getElementById("aiResponse").textContent = "";
                });
            });

            document.getElementById("startRecording").disabled = false;
            document.getElementById("stopRecording").disabled = true;


        });
    </script>
</body>
</html>


Overwriting static/index08.html


とりあえず動く形にはできた！！
ただ，どうしてもレスポンスは遅い．．．．

## その９：loggingモジュールをつかってみることにする

In [8]:
%%writefile voicecahtapp09.py
from flask import Flask, request, jsonify, send_from_directory, send_file
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading
import requests
import json
import time
import logging

#　環境変数の読み込み
load_dotenv()

# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#--------------------------------------------------
#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log"
)
#--------------------------------------------------


# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index08.html")

# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    logging.info("音声ファイルをアップロードします。")
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav")
    logging.debug(f"Saving audio file to {audio_path}")
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            logging.debug(text)
            print(text)

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

# 音声ファイルを提供するエンドポイント
@app.route("/audio/<filename>")
def get_audio(filename):
    return send_file(os.path.join("output",filename))


#--------------------------------------------------

# AIの応答を取得する関数 
def get_ai_response(text):
    
    # 現在の時刻取得
    start = time.time()

    # OpenAIのAPIを呼び出してAIの応答を取得
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})

    # 処理時間の計算
    ai_time = time.time() - start
    logging.debug(f"処理時間: {ai_time} [sec]")
    print(f"AIレスポンス時間: {ai_time} [sec]") 

    # 音声合成
    filename = synthesize_voice(ai_response)

    # 処理時間の計算
    voice_time = time.time() - start - ai_time
    logging.debug(f"音声合成時間: {voice_time} [sec]")
    print(f"音声合成時間: {voice_time} [sec]")
    
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response, 'audio': filename})



# 音声合成を行なう関数
def synthesize_voice(text, speaker=1):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'http://localhost:50021/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'http://localhost:50021/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        # 音声ファイルとして保存
        filename = f"output_{len(messages)}.wav"
        file_path = "output/" + filename
        with open(file_path, 'wb') as f:
            f.write(synthesis_response.content)
        logging.debug(f"音声が {filename} に保存されました。")
        print(f"音声が {filename} に保存されました。")
        return filename
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        print(f"Error in synthesis: {synthesis_response.text}")
        return None


if __name__ == "__main__":
    socketio.run(app, debug=True)

Overwriting voicecahtapp09.py


## その１０：htmlをチャットログが残るようにする．

In [2]:
%%writefile voicecahtapp10.py
from flask import Flask, request, jsonify, send_from_directory, send_file
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading
import requests
import json
import time
import logging

#　環境変数の読み込み
load_dotenv()

# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#--------------------------------------------------
#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)
#--------------------------------------------------


# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index10.html")

# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    logging.info("音声ファイルをアップロードします。")
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav")
    audio_file.save(audio_path)
    logging.info(f"Saved audio file to {audio_path}")

    # 音声認識
    r = sr.Recognizer()
    start_time = time.time()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    logging.info(f"音声認識結果: {text}")
    logging.info(f"音声認識時間: {time.time() - start_time} [sec]")

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

# 音声ファイルを提供するエンドポイント
@app.route("/audio/<filename>")
def get_audio(filename):
    logging.info(f"音声ファイル {filename} を返します。")
    return send_file(os.path.join("output",filename))


#--------------------------------------------------

# AIの応答を取得する関数 
def get_ai_response(text):
    
    # 現在の時刻取得
    start = time.time()

    # OpenAIのAPIを呼び出してAIの応答を取得
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")

    # 処理時間の計算
    ai_time = time.time() - start
    logging.info(f"AIレスポンス時間: {ai_time} [sec]")

    # 音声合成
    filename = synthesize_voice(ai_response)

    # 処理時間の計算
    voice_time = time.time() - start - ai_time
    logging.info(f"音声合成時間: {voice_time} [sec]")
    
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response, 'audio': filename})



# 音声合成を行なう関数
def synthesize_voice(text, speaker=1):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'http://localhost:50021/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'http://localhost:50021/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        # 音声ファイルとして保存
        filename = f"output_{len(messages)}.wav"
        file_path = "output/" + filename
        with open(file_path, 'wb') as f:
            f.write(synthesis_response.content)
        logging.info(f"音声が {filename} に保存されました。")
        print(f"音声が {filename} に保存されました。")
        return filename
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        print(f"Error in synthesis: {synthesis_response.text}")
        return None


if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)

Overwriting voicecahtapp10.py


In [ ]:
%%writefile static/index10.html

<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css">

    
</head>
<body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <div id="h_chatlog"></div>

    <script>
        let audioContext;
        let recorder;
        let audioBlob;

        document.addEventListener("DOMContentLoaded", () => {
            const socket = io();

            socket.on('ai_response', (data) => {
                const markdownText = data.ai_response;
                const htmlContent = marked.parse(markdownText);
                document.getElementById("h_chatlog").innerHTML += `<div class="assistant">${htmlContent}</div>`;
                
                // 音声ファイルを自動再生する処理
                if (data.audio) {
                    const audio = new Audio(`/audio/${data.audio}`);
                    audio.play();
                }
                document.getElementById("startRecording").disabled = false;
                document.getElementById("stopRecording").disabled = true;                    
            });

            // Spaceキーが押されたときにstartRecordingボタンをクリック
            document.addEventListener("keydown", (event) => {
                if(document.getElementById("startRecording").disabled){ 
                    console.log("処理中のため入力はできません");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    document.getElementById("startRecording").click();
                }
            });

            // Spaceキーから指が離されたときにstopRecordingボタンをクリック
            document.addEventListener("keyup", (event) => {
                if(document.getElementById("stopRecording").disabled){
                    console.log("不正な録音停止操作です");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    document.getElementById("stopRecording").click();
                }
            });

        });        

        document.getElementById("startRecording").addEventListener("click", async () => {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
        });

        document.getElementById("stopRecording").addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
                audioBlob = blob;

                if (!audioBlob) {
                    console.error("No audio to upload");    
                    return;
                }

                const formData = new FormData();
                formData.append("file", audioBlob, "recorded_audio.wav");

                fetch("/upload", {
                    method: "POST",
                    body: formData,
                })
                .then((response) => response.json())
                .then((data) => {
                    if(data.text){
                         document.getElementById("h_chatlog").innerHTML += `<div class="user">${marked.parse(data.text)}</div>`;
                    }
                    else console.log("Error: 音声を認識できませんでした。");
                })
                .catch((error) => {
                    console.error("Upload failed:");
                });
            });

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = true;
        });


    </script>
</body>
</html>


Overwriting static/index10.html


概ね上手く行くようにできた．
後の問題は，AIからのレスポンスがマークダウン形式になっているのをうまく表記することかな．

-> 対応した　marked.jsなんてのがあるんや．これ使えば簡単！

## その１１　キャラクターボイスの選択

https://zenn.dev/zenn24yykiitos/articles/f3e983fe650e08

これを参考に，フロントエンドからキャラクターを選べるようにする．

In [8]:
%%writefile voicechatapp11.py

from flask import Flask, request, jsonify, send_from_directory, send_file
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading
import requests
import json
import time
import logging

#　環境変数の読み込み
load_dotenv()

# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)

#--------------------------------------------------
# Flaskのエンドポイントの作成
#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index11.html")

# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    logging.info("音声ファイルをアップロードします。")
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    #audio_path = os.path.join("uploads", f"input_{len(messages)}.wav") #Uploadされたファイルを残すならこっちをOn
    audio_path = os.path.join("uploads", "input.wav") #Uploadされたファイルを残さないならこっちをOn
    audio_file.save(audio_path)
    logging.info(f"Saved audio file to {audio_path}")

    # 音声認識
    r = sr.Recognizer()
    start_time = time.time()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    logging.info(f"音声認識結果: {text}")
    logging.info(f"音声認識時間: {time.time() - start_time} [sec]")

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    speaker = request.form["speaker"]
    logging.info(f"AIの応答を取得します。音声合成スピーカーID: {speaker}")
    threading.Thread(target=get_ai_response, args=(text, speaker)).start()
    
    return response

# 音声ファイルを提供するエンドポイント
@app.route("/audio/<filename>")
def get_audio(filename):
    logging.info(f"音声ファイル {filename} を返します。")
    return send_file(os.path.join("output",filename))

# VoiceVoxのSpeakerIDリストを取得するエンドポイント
@app.route("/speaker_ids")
def get_speaker_ids():
    url = "http://localhost:50021/speakers"  # VOICEVOX APIのエンドポイント
    try:
        response = requests.get(url)
    except Exception as e:
        logging.error(f"Error: {e}")
        return jsonify([])

    voicevox_speakers = []
    if response.status_code == 200:
        speakers = response.json()
        for speaker in speakers:
            name = speaker['name']
            style_names = [style['name'] for style in speaker['styles']]
            style_ids = [style['id'] for style in speaker['styles']]
            for style_id, style_name in zip(style_ids, style_names):
                voicevox_speakers.append(f"<option value={style_id}>Speaker: {name}, {style_name} </option>")
        logging.info("speaker_ids を取得しました。")
        return jsonify(voicevox_speakers)
    else:
        logging.error(f"Error: {response.status_code}")
        return jsonify([])    

# VoiceVoxの音声テストを行うエンドポイント
@app.route("/speaker_test" , methods=["POST"])
def speaker_test():
    speaker = request.json["speaker"]
    text = "こんにちは．初めまして．何かお手伝いできることはありますか？"
    filename = synthesize_voice(text, speaker)
    return jsonify({"audio": filename})


#--------------------------------------------------
# Flaskの各エンドポイント内の処理関数
#--------------------------------------------------
# AIの応答を取得する関数 
def get_ai_response(text, speaker):
    
    # 現在の時刻取得
    start = time.time()

    # OpenAIのAPIを呼び出してAIの応答を取得
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")

    # 処理時間の計算
    ai_time = time.time() - start
    logging.info(f"AIレスポンス時間: {ai_time} [sec]")

    # 音声合成
    filename = synthesize_voice(ai_response, speaker)

    # 処理時間の計算
    voice_time = time.time() - start - ai_time
    logging.info(f"音声合成時間: {voice_time} [sec]")
    
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response, 'audio': filename})



# VoiceVox APIで音声合成を行なう関数
def synthesize_voice(text, speaker):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'http://localhost:50021/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'http://localhost:50021/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        # 音声ファイルとして保存
        #filename = f"output_{len(messages)}.wav" #合成音声を全部残すならこっちをON
        filename = "output.wav" #合成音声を全部残さないならこっちをON
        file_path = "output/" + filename
        with open(file_path, 'wb') as f:
            f.write(synthesis_response.content)
        logging.info(f"音声が {filename} に保存されました。")
        return filename
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None


if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)


Overwriting voicechatapp11.py


In [ ]:
%%writefile static/index11.html
<html lang="ja">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css" />
  </head>

  <body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <select id="h_speakerSelect"></select>
    <button id="speakerTest">音声テスト</button>
    <div id="h_chatlog"></div>

    <script>
      navigator.mediaDevices
        .getUserMedia({ audio: true })
        .then((stream) => {
          window.stream = stream;
        })
        .catch((error) => {
          console.error("Error accessing the microphone: " + error);
        });
      let audioContext;
      let recorder;
      let audioBlob;

      document.addEventListener("DOMContentLoaded", () => {
        const socket = io();

        // SpeakerIDリストを取得
        fetch("/speaker_ids")
          .then((response) => response.json())
          .then((data) => {
            const h_speakerSelect = document.getElementById("h_speakerSelect");
            h_speakerSelect.innerHTML = data.join("");
          });

        // AIの応答を受信したときの処理
        socket.on("ai_response", (data) => {
          const markdownText = data.ai_response;
          const htmlContent = marked.parse(markdownText);
          document.getElementById(
            "h_chatlog"
          ).innerHTML += `<div class="assistant">${htmlContent}</div>`;

          // 音声ファイルを自動再生する処理
          console.log("audioリクエスト");
          const audio = new Audio(`/audio/${data.audio}`);
          console.log("audio受信しました", audio);
          audio.play();

          document.getElementById("startRecording").disabled = false;
          document.getElementById("stopRecording").disabled = true;
          document.getElementById("selectSpeaker").disabled = false;
        });

        // Spaceキーが押されたときにstartRecordingボタンをクリック
        document.addEventListener("keydown", (event) => {
          if (document.getElementById("startRecording").disabled) {
            console.log("処理中のため入力はできません");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            document.getElementById("startRecording").click();
          }
        });

        // Spaceキーから指が離されたときにstopRecordingボタンをクリック
        document.addEventListener("keyup", (event) => {
          if (document.getElementById("stopRecording").disabled) {
            console.log("不正な録音停止操作です");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            document.getElementById("stopRecording").click();
          }
        });

        document.getElementById("speakerTest").addEventListener("click", () => {
          const speaker = document.getElementById("h_speakerSelect").value;
          fetch("/speaker_test", {
            method: "POST",
            headers: {
              "Content-Type": "application/json",
            },
            body: JSON.stringify({ speaker }),
          })
            .then((response) => response.json())
            .then((data) => {
              const audio = new Audio(`/audio/${data.audio}`);
              audio.play();
            });
        });

        document
          .getElementById("startRecording")
          .addEventListener("click", () => {
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(window.stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
            document.getElementById("selectSpeaker").disabled = true;
          });

        document
          .getElementById("stopRecording")
          .addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
              audioBlob = blob;

              if (!audioBlob) {
                console.error("No audio to upload");
                return;
              }

              const formData = new FormData();
              formData.append("file", audioBlob, "recorded_audio.wav");

              const speaker = document.getElementById("h_speakerSelect").value;
              formData.append("speaker", speaker);

              fetch("/upload", {
                method: "POST",
                body: formData,
              })
                .then((response) => response.json())
                .then((data) => {
                  if (data.text) {
                    document.getElementById(
                      "h_chatlog"
                    ).innerHTML += `<div class="user">${marked.parse(
                      data.text
                    )}</div>`;
                  } else console.log("Error: 音声を認識できませんでした。");
                })
                .catch((error) => {
                  console.error("Upload failed:");
                });
            });

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = true;
          });
      });
    </script>
  </body>
</html>

Overwriting static/index11.html


## その１２　よりリアルタイム化したい
テキストを句点や感嘆符などの句単位に区切り，それをflask側でストリーミングで音声合成し，フロントエンドにストリーミで返す．フロントエンドでは受け取ったストリームを再生する．

In [ ]:
from flask import Flask, request, Response, jsonify, send_from_directory, send_file, stream_with_context
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading
import requests
import json
import time
import logging
import re

#　環境変数の読み込み
load_dotenv()

# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)

#--------------------------------------------------
# Flaskのエンドポイントの作成
#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index12.html")

# VoiceVoxのSpeakerIDリストを取得するエンドポイント
@app.route("/speaker_ids")
def get_speaker_ids():
    url = "http://localhost:50021/speakers"  # VOICEVOX APIのエンドポイント
    try:
        response = requests.get(url)
    except Exception as e:
        logging.error(f"Error: {e}")
        return jsonify([])

    voicevox_speakers = []
    if response.status_code == 200:
        speakers = response.json()
        for speaker in speakers:
            name = speaker['name']
            style_names = [style['name'] for style in speaker['styles']]
            style_ids = [style['id'] for style in speaker['styles']]
            for style_id, style_name in zip(style_ids, style_names):
                voicevox_speakers.append(f"<option value={style_id}>Speaker: {name}, {style_name} </option>")
        logging.info("speaker_ids を取得しました。")
        return jsonify(voicevox_speakers)
    else:
        logging.error(f"Error: {response.status_code}")
        return jsonify([])    

# VoiceVoxの音声テストを行うエンドポイント
@app.route("/speaker_test" , methods=["POST"])
def speaker_test():
    speaker = request.json["speaker"]
    text = "こんにちは．初めまして．何かお手伝いできることはありますか？"
    filename = synthesize_voice(text, speaker)
    return jsonify({"audio": filename})


# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav") #Uploadされたファイルを残すならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    
    # AIの応答を取得
    ai_response = get_ai_response(text)
    ## WebSocketを通じてクライアントに通知
    if ai_response:
        socketio.emit('ai_response', {'ai_response': ai_response}) #, 'audio': filename})
    else:
        return jsonify({"error": "Failed to get AI response"}), 400

    # AIの応答から音声合成
    speaker = request.form["speaker"]
    filename = synthesize_voice(ai_response, speaker)
    ## WebSocketを通じてクライアントに通知
    if filename:
        socketio.emit('play_audio', {'audio': filename})
    else:
        return jsonify({"error": "Failed to synthesize voice"}), 400
    
    return jsonify({"info": "Process Succeeded"}), 200


# 音声ファイルを提供するエンドポイント
@app.route("/audio/<filename>")
def get_audio(filename):
    return send_file(os.path.join("output",filename))



# streaming処理するエンドポイント
@app.route("/streaming", methods=["POST"])
def streaming():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", "input.wav") #Uploadされたファイルを残さないならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    # 音声認識の結果を最初に返す
    socketio.emit("SpeechRecognition",{"text": text})
    
    # 続けてストリームで音声合成
    ## まずは openai で応答を取得
    ai_response = get_ai_response(text)
    socketio.emit("AIResponse", {"ai_response": ai_response})

    ## 音声合成
    speaker = request.form["speaker"]
    logging.info(f"AIの応答を取得します。音声合成スピーカーID: {speaker}")    
    

    def generate():
        yield from synthesize_streaming(ai_response, speaker)

    socketio.emit("Streaming", {
        "Response": Response(
            stream_with_context(generate()),
            content_type="application/octet-stream"
        )
    })
    
    return jsonify({"info": "Process Succeeded"}), 200





#--------------------------------------------------
# Flaskの各エンドポイント内の処理関数
#--------------------------------------------------
# 音声認識を行う関数
def recognize_speech(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    return text

# OpenAIのAPIを呼び出してAIの応答を取得する関数
def get_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")
    return ai_response


# VoiceVox APIで音声合成を行なう関数
def synthesize_voice(text, speaker):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'http://localhost:50021/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'http://localhost:50021/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        # 音声ファイルとして保存
        filename = f"output_{len(messages)}.wav" #合成音声を全部残すならこっちをON
        #filename = "output.wav" #合成音声を全部残さないならこっちをON
        file_path = "output/" + filename
        with open(file_path, 'wb') as f:
            f.write(synthesis_response.content)
        logging.info(f"音声が {filename} に保存されました。")
        return filename
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None


# テキストを句単位に区切る
def preprocess_text(text):
    # テキストの前処理
    text = re.sub(r"[。．.]", "。\n", text)
    text = re.sub(r"[？?]", "？\n", text)
    text = re.sub(r"[！!]", "！\n", text)
    return text

# テキストを句ごとに音声合成してストリーミング
def synthesize_streaming(text, speaker):
    # テキストを句単位に区切る
    logging.debug("テキストを句単位に区切ります。")
    text = preprocess_text(text)
    sentences = text.split("\n")

    # 句ごとに音声合成
    for sentence in sentences:
        if sentence == "": continue
        
        ## クエリ
        query_response = requests.post(
            f'http://localhost:50021/audio_query', 
            params={'text': sentence, 'speaker': speaker}
        )
        if query_response.status_code != 200:
            logging.error(f"Error in audio_query: {query_response.text}")
            return
        
        ## 音声合成
        logging.debug("音声データを生成します。")
        with requests.post(
            f'http://localhost:50021/synthesis', 
            params={'speaker': speaker}, 
            json=query_response.json(), 
            stream=True
        ) as synthesis_response:
            if synthesis_response.status_code != 200:
                logging.error(f"Error in synthesis: {synthesis_response.text}")
                return
            yield "---start---\n".encode("utf-8")
            for chunk in synthesis_response.iter_content(chunk_size=1024):
                logging.info("チャンク生成")
                yield chunk
            yield "---end---\n".encode("utf-8")

            time.sleep(0.2)
        
    


if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)


Writing voicechatapp12.py


In [ ]:
%%writefile static/index12.html
<html lang="ja">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css" />
  </head>

  <body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording">録音停止</button>
    <button id="stopRecordingWithStreaming">停止とストリーミング処理</button>
    <select id="h_speakerSelect"></select>
    <button id="speakerTest">音声テスト</button>
    <div id="h_chatlog"></div>

    <script>
      navigator.mediaDevices
        .getUserMedia({ audio: true })
        .then((stream) => {
          window.stream = stream;
        })
        .catch((error) => {
          console.error("Error accessing the microphone: " + error);
        });
      let audioContext;
      let recorder;
      let audioBlob;

      // フォーム要素取得
      const h_startRecButton = document.getElementById("startRecording");
      const h_stopRecButton = document.getElementById("stopRecording");
      const stopRecwithStreamingButton = document.getElementById(
        "stopRecordingWithStreaming"
      );
      const h_speakerSelect = document.getElementById("h_speakerSelect");
      const h_speakerTestButton = document.getElementById("speakerTest");

      // 録音開始時のボタンを無効化
      function setBtnonStart() {
        h_startRecButton.disabled = true;
        h_stopRecButton.disabled = false;
        stopRecwithStreamingButton.disabled = false;
        h_speakerSelect.disabled = true;
        h_speakerTestButton.disabled = true;
      }

      // 処理中のボタン無効化
      function setBtnunderProcessing() {
        h_startRecButton.disabled = true;
        h_stopRecButton.disabled = true;
        stopRecwithStreamingButton.disabled = true;
        h_speakerSelect.disabled = true;
        h_speakerTestButton.disabled = true;
      }

      // 復帰時のボタン有効化
      function setBtnonRestart() {
        h_startRecButton.disabled = false;
        h_stopRecButton.disabled = true;
        stopRecwithStreamingButton.disabled = true;
        h_speakerSelect.disabled = false;
        h_speakerTestButton.disabled = false;
      }

      // 音声合成のストリーミング処理
      async function playSnetence(chunks, audioContext) {
        const combined = new Uint8Array(
          chunks.reduce((acc, chunk) => [...acc, ...chunk], [])
        );
        const audioBuffer = await audioContext.decodeAudioData(combined.buffer);
        const source = audioContext.createBufferSource();
        source.buffer = audioBuffer;
        source.connect(audioContext.destination);
        source.start();

        await new Promise((resolve) => {
          source.onended = resolve;
        });
      }

      document.addEventListener("DOMContentLoaded", () => {
        const socket = io();

        // SpeakerIDリストを取得
        fetch("/speaker_ids")
          .then((response) => response.json())
          .then((data) => {
            h_speakerSelect.innerHTML = data.join("");
          });

        // 音声認識の結果を受信
        socket.on("SpeechRecognition", (data) => {
          const markdownText = data.text;
          const htmlContent = marked.parse(markdownText);
          document.getElementById(
            "h_chatlog"
          ).innerHTML += `<div class="user">${htmlContent}</div>`;
        });

        // AIの応答を受信したときの処理
        socket.on("ai_response", (data) => {
          const markdownText = data.ai_response;
          const htmlContent = marked.parse(markdownText);
          document.getElementById(
            "h_chatlog"
          ).innerHTML += `<div class="assistant">${htmlContent}</div>`;
        });

        // 音声ファイルを再生する処理
        socket.on("play_audio", (data) => {
          const audio = new Audio(`/audio/${data.audio}`);
          audio.play();
        });

        //Stremingで音声合成の結果を受信
        socket.on("Streaming", async (data) => {
          const reader = data.Response.body.getReader();
          const audioContext = new AudioContext();
          let chunks = [];
          let proccesing = false;

          while (true) {
            const { done, value } = await reader.read();
            if (done) break;
            const textChunk = new TextDecorder("utf-8").decode(value);
            if (textChunk.include("---start---")) {
              if (chunks.length > 0 && !proccesing) {
                proccesing = true;
                await playSnetence(chunks, audioContext);
                chunks = [];
                proccesing = false;
              }
            } else if (textChunk.include("---end---")) {
              proccesing = true;
              await playSnetence(chunks, audioContext);
              chunks = [];
              proccesing = false;
            } else {
              chunks.push(value);
            }
          }
        });

        // Spaceキーが押されたときにstartRecordingボタンをクリック
        document.addEventListener("keydown", (event) => {
          if (h_startRecButton.disabled) {
            console.log("処理中のため入力はできません");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            h_startRecButton.click();
          }
        });

        // Spaceキーから指が離されたときにstopRecordingボタンをクリック
        document.addEventListener("keyup", (event) => {
          if (h_stopRecButton.disabled) {
            console.log("不正な録音停止操作です");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            h_stopRecButton.click();
          }
        });

        //Speakerの音声確認テスト
        h_speakerTestButton.addEventListener("click", () => {
          const speaker = h_speakerSelect.value;
          fetch("/speaker_test", {
            method: "POST",
            headers: {
              "Content-Type": "application/json",
            },
            body: JSON.stringify({ speaker }),
          })
            .then((response) => response.json())
            .then((data) => {
              const audio = new Audio(`/audio/${data.audio}`);
              audio.play();
            });
        });

        // 録音開始ボタンがクリックされたときの処理
        h_startRecButton.addEventListener("click", () => {
          audioContext = new AudioContext();
          const source = audioContext.createMediaStreamSource(window.stream);
          recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
          recorder.record();

          // ボタンを無効化
          setBtnonStart();
        });

        // 録音停止ボタンがクリックされたときの処理
        h_stopRecButton.addEventListener("click", () => {
          // ボタンを無効化
          setBtnunderProcessing();

          // 録音を停止
          recorder.stop();

          // 録音した音声をファイルに保存して送信
          recorder.exportWAV((blob) => {
            audioBlob = blob;

            if (!audioBlob) {
              console.error("No audio to upload");
              return;
            }

            const formData = new FormData();
            formData.append("file", audioBlob, "recorded_audio.wav");

            const speaker = h_speakerSelect.value;
            formData.append("speaker", speaker);

            fetch("/upload", {
              method: "POST",
              body: formData,
            })
              .then((response) => response.json())
              .then((data) => {
                console.log(data);
                // ボタン状態の初期化
                setBtnonRestart();
              })
              .catch((error) => {
                console.error("Upload failed:");
                // ボタン状態の初期化
                setBtnonRestart();
              });
          });
        });

        stopRecwithStreamingButton.addEventListener("click", () => {
          // ボタンを無効化
          setBtnunderProcessing();

          // 録音を停止
          recorder.stop();

          // 録音した音声をファイルに保存して送信
          recorder.exportWAV((blob) => {
            audioBlob = blob;

            if (!audioBlob) {
              console.error("No audio to upload");
              return;
            }

            const formData = new FormData();
            formData.append("file", audioBlob, "recorded_audio.wav");

            const speaker = h_speakerSelect.value;
            formData.append("speaker", speaker);

            fetch("/streaming", {
              method: "POST",
              body: formData,
            })
              .then((response) => response.json())
              .then((data) => {
                console.log(data);
                // ボタン状態の初期化
                setBtnonRestart();
              })
              .catch((error) => {
                console.error("Upload failed:");
                // ボタン状態の初期化
                setBtnonRestart();
              });
          });
        });

        // ボタン状態の初期化
        setBtnonRestart();
      });
    </script>
  </body>
</html>


Writing static/index12.html


まだストリーム再生はできていない．
けど，ソケット通信でPush型で動くようにした．
プログラム的にはだいぶ綺麗にはなったとは思う．

## その１３　改めて音声をストリームで受け取る形にする
GPTの出力を文ごとに切り分けて，1文ずつVoice Voxに送り，合成された音声を順次mp３でクライアントに送る．
クライアントでは受け取った音声を順次キューに入れていく．

In [ ]:
%%writefile voicechatapp13.py

from flask import Flask, request, Response, jsonify, send_from_directory, send_file, stream_with_context
from flask_cors import CORS
from flask_socketio import SocketIO, emit
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
import requests
import time
import logging
import re
from pydub import AudioSegment
from io import BytesIO

#　環境変数の読み込み
load_dotenv()

# VoiceVox APIのエンドポイント
VOICEVOX_API_URL = "http://localhost:50021"


# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)

#--------------------------------------------------
# Flaskのエンドポイントの作成
#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index13.html")

# VoiceVoxのSpeakerIDリストを取得するエンドポイント
@app.route("/speaker_ids")
def get_speaker_ids():
    url = f"{VOICEVOX_API_URL}/speakers"  # VOICEVOX APIのエンドポイント
    try:
        response = requests.get(url)
    except Exception as e:
        logging.error(f"Error: {e}")
        return jsonify([])

    voicevox_speakers = []
    if response.status_code == 200:
        speakers = response.json()
        for speaker in speakers:
            name = speaker['name']
            style_names = [style['name'] for style in speaker['styles']]
            style_ids = [style['id'] for style in speaker['styles']]
            for style_id, style_name in zip(style_ids, style_names):
                voicevox_speakers.append(f"<option value={style_id}>Speaker: {name}, {style_name} </option>")
        logging.info("speaker_ids を取得しました。")
        return jsonify(voicevox_speakers)
    else:
        logging.error(f"Error: {response.status_code}")
        return jsonify([])    

# VoiceVoxの音声テストを行うエンドポイント
@app.route("/speaker_test" , methods=["POST"])
def speaker_test():
    speaker = request.json["speaker"]
    text = "こんにちは．初めまして．何かお手伝いできることはありますか？"
    synthesize_response = synthesize_voice(ai_response, speaker)

    # 合成した音声をmp3化
    if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
    mp3_data  = BytesIO()
    audio.export(mp3_data , format="mp3")
    mp3_data .seek(0)  

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
    return jsonify({"info": "Speaker Test Process Succeeded"}), 200


# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav") #Uploadされたファイルを残すならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    
    # AIの応答を取得
    ai_response = get_ai_response(text)
    ## WebSocketを通じてクライアントに通知
    if ai_response:
        socketio.emit('ai_response', {'ai_response': ai_response}) #, 'audio': filename})
    else:
        return jsonify({"error": "Failed to get AI response"}), 400

    # AIの応答から音声合成
    speaker = request.form["speaker"]
    synthesize_response = synthesize_voice(ai_response, speaker)


    # 合成した音声をmp3化
    if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
    mp3_data  = BytesIO()
    audio.export(mp3_data , format="mp3")
    mp3_data .seek(0)  

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
    return jsonify({"info": "Uploard Process Succeeded"}), 200



# streaming処理するエンドポイント
@app.route("/streaming", methods=["POST"])
def streaming():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", "input.wav") #Uploadされたファイルを残さないならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    
    # AIの応答を取得
    ai_response = get_ai_response(text)
    ## WebSocketを通じてクライアントに通知
    if ai_response:
        socketio.emit('ai_response', {'ai_response': ai_response}) #, 'audio': filename})
    else:
        return jsonify({"error": "Failed to get AI response"}), 400
    

    ## 音声合成
    speaker = request.form["speaker"]

    # テキストを句単位に区切る
    logging.debug("テキストを句単位に区切ります。")
    text = preprocess_text(ai_response)
    sentences = text.split("\n")

    # 句ごとに音声合成
    for sentence in sentences:
        if sentence == "": continue
        
        ## 音声合成
        synthesize_response=synthesize_voice(sentence, speaker)
        if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400

        ## 合成した音声をmp3化
        audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
        mp3_data  = BytesIO()
        audio.export(mp3_data , format="mp3")
        mp3_data .seek(0)

        ## mp3データをWebSocketを通じてクライアントに通知 ここでうまくキューに入れて連続再生させたい
        socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
        time.sleep(0.5)

    
    return jsonify({"info": "Process Succeeded"}), 200


#--------------------------------------------------
# Flaskの各エンドポイント内の処理関数
#--------------------------------------------------
# 音声認識を行う関数
def recognize_speech(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    return text

# OpenAIのAPIを呼び出してAIの応答を取得する関数
def get_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")
    return ai_response


# VoiceVox APIで音声合成を行なう関数
def synthesize_voice(text, speaker):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'{VOICEVOX_API_URL}/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'{VOICEVOX_API_URL}/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        logging.info("音声データを生成しました。")
        return synthesis_response
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None


# テキストを句単位に区切る
def preprocess_text(text):
    # テキストの前処理
    text = re.sub(r"[。．.]", "。\n", text)
    text = re.sub(r"[？?]", "？\n", text)
    text = re.sub(r"[！!]", "！\n", text)
    return text


if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)


In [ ]:
%%witefile /static/index13.html
<html lang="ja">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>WAV録音アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css" />
  </head>

  <body>
    <h1>WAV録音アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording">録音停止</button>
    <input type="radio" id="streaming" name="Method" value="/upload" checked>まとめて再生(基本)</radio>
    <input type="radio" id="streaming" name="Method" value="/streaming" >ストリーミング</radio>
    <select id="h_speakerSelect"></select>
    <button id="speakerTest">音声テスト</button>
    <div id="h_chatlog"></div>

    <script>
      navigator.mediaDevices
        .getUserMedia({ audio: true })
        .then((stream) => {
          window.stream = stream;
        })
        .catch((error) => {
          console.error("Error accessing the microphone: " + error);
        });

      let audioContext;
      let recorder;
      let audioBlob;
      let audioQueue = [];
      let isPlaying = false;

      // フォーム要素取得
      const h_startRecButton = document.getElementById("startRecording");
      const h_stopRecButton = document.getElementById("stopRecording");
      const h_speakerSelect = document.getElementById("h_speakerSelect");
      const h_speakerTestButton = document.getElementById("speakerTest");

      // 録音開始時のボタンを無効化
      function setBtnonStart() {
        h_startRecButton.disabled = true;
        h_stopRecButton.disabled = false;
        h_speakerSelect.disabled = true;
        h_speakerTestButton.disabled = true;
      }

      // 処理中のボタン無効化
      function setBtnunderProcessing() {
        h_startRecButton.disabled = true;
        h_stopRecButton.disabled = true;
        h_speakerSelect.disabled = true;
        h_speakerTestButton.disabled = true;
      }

      // 復帰時のボタン有効化
      function setBtnonRestart() {
        h_startRecButton.disabled = false;
        h_stopRecButton.disabled = true;
        h_speakerSelect.disabled = false;
        h_speakerTestButton.disabled = false;
      }


      document.addEventListener("DOMContentLoaded", () => {
        const socket = io();

        // SpeakerIDリストを取得
        fetch("/speaker_ids")
          .then((response) => response.json())
          .then((data) => {
            h_speakerSelect.innerHTML = data.join("");
          });

        // 音声認識の結果を受信
        socket.on("SpeechRecognition", (data) => {
          const markdownText = data.text;
          const htmlContent = marked.parse(markdownText);
          document.getElementById(
            "h_chatlog"
          ).innerHTML += `<div class="user">${htmlContent}</div>`;
        });

        // AIの応答を受信したときの処理
        socket.on("ai_response", (data) => {
          const markdownText = data.ai_response;
          const htmlContent = marked.parse(markdownText);
          document.getElementById(
            "h_chatlog"
          ).innerHTML += `<div class="assistant">${htmlContent}</div>`;
        });

        // 音声ファイルを再生する処理
        socket.on("play_audio", async(data) => {
            const audioBlob = new Blob([data.audio], { type: "audio/mp3" });
            const audioUrl = URL.createObjectURL(audioBlob);

            // キューに登録
            audioQueue.push(audioUrl);

            // 再生中でなければ再生
            if (!isPlaying) {
                playAudio();
            }
            // const audio = new Audio(audioUrl);
            // audio.play();
        });

        // Queueに登録された音声ファイルを再生する処理
        async function playAudio() {
            // 再生する音声ファイルがなければ終了
            if (audioQueue.length === 0) {
                isPlaying = false;
                return;
            }

            isPlaying = true;
            const audioUrl = audioQueue.shift();
            const audio = new Audio(audioUrl);
            audio.play();

            // 再生が終了したら次の音声ファイルを再生
            audio.onended = () => {
                playAudio();
            };
        }

        // //Stremingで音声合成の結果を受信
        // socket.on("Streaming", async (data) => {

        // });

        // Spaceキーが押されたときにstartRecordingボタンをクリック
        document.addEventListener("keydown", (event) => {
          if (h_startRecButton.disabled) {
            console.log("処理中のため入力はできません");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            h_startRecButton.click();
          }
        });

        // Spaceキーから指が離されたときにstopRecordingボタンをクリック
        document.addEventListener("keyup", (event) => {
          if (h_stopRecButton.disabled) {
            console.log("不正な録音停止操作です");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            h_stopRecButton.click();
          }
        });

        //Speakerの音声確認テスト
        h_speakerTestButton.addEventListener("click", () => {
          const speaker = h_speakerSelect.value;
          fetch("/speaker_test", {
            method: "POST",
            headers: {
              "Content-Type": "application/json",
            },
            body: JSON.stringify({ speaker }),
          })
            .then((response) => response.json())
            .then((data) => {
              console.log(data);
            });
        });

        // 録音開始ボタンがクリックされたときの処理
        h_startRecButton.addEventListener("click", () => {
          audioContext = new AudioContext();
          const source = audioContext.createMediaStreamSource(window.stream);
          recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
          recorder.record();

          // ボタンを無効化
          setBtnonStart();
        });

        // 録音停止ボタンがクリックされたときの処理
        h_stopRecButton.addEventListener("click", () => {
          // ボタンを無効化
          setBtnunderProcessing();

          // 録音を停止
          recorder.stop();

          // 録音した音声をファイルに保存して送信
          recorder.exportWAV((blob) => {
            audioBlob = blob;
            if (!audioBlob) {
              console.error("No audio to upload");
              return;
            }

            const formData = new FormData();
            formData.append("file", audioBlob, "recorded_audio.wav");

            const speaker = h_speakerSelect.value;
            formData.append("speaker", speaker);

            const method = document.querySelector('input[name="Method"]:checked').value;

            fetch(method, {
              method: "POST",
              body: formData,
            })
              .then((response) => response.json())
              .then((data) => {
                console.log(data);
                // ボタン状態の初期化
                setBtnonRestart();
              })
              .catch((error) => {
                console.error("Upload failed:");
                // ボタン状態の初期化
                setBtnonRestart();
              });
          });
        });

        // ボタン状態の初期化
        setBtnonRestart();
      });

      // ページを離れるときにストリームを停止
      window.addEventListener("beforeunload", () => {
        if (window.stream) {
          window.stream.getTracks().forEach((track) => {
            track.stop();
          });
        }
      });
    </script>
  </body>
</html>


だいぶ苦労したけど，少しずつ形になってきた．
このあとはGPTからの返答もストリーミングで受け取れるようにする．

## その１４　GPTからの返答をストリームで受け取る．
これの場合，GPTからのストリーム出力を一旦キープして文末に来たところで一気に処理をかけるという処理になる．

In [ ]:
%%writefile voicechatapp14.py

from flask import Flask, request, Response, jsonify, send_from_directory, send_file, stream_with_context
from flask_cors import CORS
from flask_socketio import SocketIO, emit
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
import requests
import time
import logging
import re
from pydub import AudioSegment
from io import BytesIO

#　環境変数の読み込み
load_dotenv()

# VoiceVox APIのエンドポイント
VOICEVOX_API_URL = "http://localhost:50021"


# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)

#--------------------------------------------------
# Flaskのエンドポイントの作成
#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index14.html")

# VoiceVoxのSpeakerIDリストを取得するエンドポイント
@app.route("/speaker_ids")
def get_speaker_ids():
    url = f"{VOICEVOX_API_URL}/speakers"  # VOICEVOX APIのエンドポイント
    try:
        response = requests.get(url)
    except Exception as e:
        logging.error(f"Error: {e}")
        return jsonify([])

    voicevox_speakers = []
    if response.status_code == 200:
        speakers = response.json()
        for speaker in speakers:
            name = speaker['name']
            style_names = [style['name'] for style in speaker['styles']]
            style_ids = [style['id'] for style in speaker['styles']]
            for style_id, style_name in zip(style_ids, style_names):
                voicevox_speakers.append(f"<option value={style_id}>Speaker: {name}, {style_name} </option>")
        logging.info("speaker_ids を取得しました。")
        return jsonify(voicevox_speakers)
    else:
        logging.error(f"Error: {response.status_code}")
        return jsonify([])    

# VoiceVoxの音声テストを行うエンドポイント
@app.route("/speaker_test" , methods=["POST"])
def speaker_test():
    speaker = request.json["speaker"]
    text = "こんにちは．初めまして．何かお手伝いできることはありますか？"
    synthesize_response = synthesize_voice(ai_response, speaker)

    # 合成した音声をmp3化
    if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
    mp3_data  = BytesIO()
    audio.export(mp3_data , format="mp3")
    mp3_data .seek(0)  

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
    return jsonify({"info": "Speaker Test Process Succeeded"}), 200


# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav") #Uploadされたファイルを残すならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    
    # # AIの応答をストリームで生成
    # socketio.emit('ai_response', {'ai_response': "---Start---"}) # 開始を通知
    # for ai_response in generate_ai_response(text):
    #     ## WebSocketを通じてクライアントに通知
    #     if ai_response:
    #         socketio.emit('ai_response', {'ai_response': ai_response}) 
    #     else:
    #         return jsonify({"error": "Failed to get AI response"}), 400
    # socketio.emit('ai_response', {'ai_response': "---End---"}) # 終了を通知

    # AIの応答を取得
    ai_response = get_ai_response(text)
    ## WebSocketを通じてクライアントに通知
    if ai_response:
        socketio.emit('ai_response', {'ai_response': ai_response}) 
    else:
        return jsonify({"error": "Failed to get AI response"}), 400
    
    # AIの応答から音声合成
    speaker = request.form["speaker"]
    synthesize_response = synthesize_voice(ai_response, speaker)


    # 合成した音声をmp3化
    if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
    mp3_data  = BytesIO()
    audio.export(mp3_data , format="mp3")
    mp3_data .seek(0)  

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})

    return jsonify({"info": "Uploard Process Succeeded"}), 200



# streaming処理するエンドポイント
@app.route("/streaming", methods=["POST"])
def streaming():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", "input.wav") #Uploadされたファイルを残さないならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    

    # AIの応答を句単位でストリームするとともに．句単位で音声合成もしていく
    """
    現状では，多分句の表示と音声が同期しない．句は順に表示されていくけど，音声はキューに入って順に再生されるので．
    これを同期させようと思うと，Javascriptに句を送ったものも一旦キューに入れて，音声と句を同時に処理するようにしないといけな
    い．
    できなくはないか・・・
    """
    speaker = request.form["speaker"]
    socketio.emit('ai_stream', {'ai_stream': "---Start---"}) # 開始を通知
    for sentence in generate_ai_response(text):
        ## WebSocketを通じてクライアントに通知
        if sentence:
            # 音声合成
            synthesize_response=synthesize_voice(sentence, speaker)
            if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
            ## 合成した音声をmp3化
            audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
            mp3_data  = BytesIO()
            audio.export(mp3_data , format="mp3")
            mp3_data .seek(0)
            ## mp3データをWebSocketを通じてクライアントに通知 ここでうまくキューに入れて連続再生させたい
            socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
            socketio.emit('ai_stream', {'ai_stream': sentence})
            ## 0.5秒の無音を入れる．これで句の切り分けが聞きやすくなると思う．
            silent_audio = AudioSegment.silent(duration=500)
            mp3_data  = BytesIO()
            silent_audio.export(mp3_data , format="mp3")
            mp3_data .seek(0)
            socketio.emit('play_audio', {'audio': mp3_data.getvalue()}) 
        else:
            return jsonify({"error": "Failed to get AI response"}), 400
    socketio.emit('ai_stream', {'ai_stream': "---End---"}) # 終了を通知

    
    return jsonify({"info": "Process Succeeded"}), 200


#--------------------------------------------------
# Flaskの各エンドポイント内の処理関数
#--------------------------------------------------
# 音声認識を行う関数
def recognize_speech(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    return text

# OpenAIのAPIを呼び出してAIの応答を取得する関数
def get_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")
    return ai_response

# OpenAIのAPIを呼び出してAIの応答をストリームで生成する関数
def generate_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream = True
    )

    sentens = "" # 句を構成するためのバッファ
    message = "" # プロンプトに含めるためにチャンクを結合させるためのためのバッファ
    for chunk in completion:
        # きちんとしたチャンクが帰ってきているかのチェック
        if "choices" in chunk.to_dict() and len(chunk.choices) > 0: #to_dict：辞書型に変えないと”choices”が見つからないようなので
            content  = chunk.choices[0].delta.content
            if content:
                message += content
                # 1文字ずつ取り出してチェックする
                for i in range(len(content)):
                    char = content[i]
                    sentens += char
                    if char in "。．.？?！!\n": #今見ているのが区切り文字だった場合
                        if i < len(content)-1: # i が最後の文字でないなら，次の文字をチェック
                            if content[i+1] not in "。．.？?！!\n": #次の文字が区切り文字でないならyield
                                logging.debug(f"句: {sentens}")
                                yield sentens
                                sentens = ""
                            else: #もし次の文字が区切り文字なら，現時点の区切り文字はスルー
                                continue
                        else: #iが最後の文字の場合，現時点でyield
                            logging.debug(f"句: {sentens}")
                            yield sentens
                            sentens = ""
    # 最後の句を返す
    if sentens:
        yield sentens
    
    # message をmessagesに追加
    messages.append({"role": "assistant", "content": message})
    logging.info(f"AIの応答: {message}")



# VoiceVox APIで音声合成を行なう関数
def synthesize_voice(text, speaker):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'{VOICEVOX_API_URL}/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'{VOICEVOX_API_URL}/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        logging.info("音声データを生成しました。")
        return synthesis_response
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None


# テキストを句単位に区切る
def preprocess_text(text):
    # テキストの前処理
    text = re.sub(r"[。．.]", "。\n", text)
    text = re.sub(r"[？?]", "？\n", text)
    text = re.sub(r"[！!]", "！\n", text)
    return text


if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)


Writing voicechatapp14.py


In [ ]:
%%writefile /static/index14.html
<html lang="ja">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>WAV録音アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css" />
  </head>

  <body>
    <h1>WAV録音アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording">録音停止</button>
    <input type="radio" id="streaming" name="Method" value="/upload" checked>まとめて再生(基本)</radio>
    <input type="radio" id="streaming" name="Method" value="/streaming" >ストリーミング</radio>
    <select id="h_speakerSelect"></select>
    <button id="speakerTest">音声テスト</button>
    <div id="h_chatlog"></div>

    <script>
      navigator.mediaDevices
        .getUserMedia({ audio: true })
        .then((stream) => {
          window.stream = stream;
        })
        .catch((error) => {
          console.error("Error accessing the microphone: " + error);
        });

      let audioContext;
      let recorder;
      let audioBlob;
      let audioQueue = [];
      let isPlaying = false;

      // html要素取得
      const h_startRecButton = document.getElementById("startRecording");
      const h_stopRecButton = document.getElementById("stopRecording");
      const h_speakerSelect = document.getElementById("h_speakerSelect");
      const h_speakerTestButton = document.getElementById("speakerTest");
      const h_chatlog = document.getElementById("h_chatlog");

      // 録音開始時のボタンを無効化
      function setBtnonStart() {
        h_startRecButton.disabled = true;
        h_stopRecButton.disabled = false;
        h_speakerSelect.disabled = true;
        h_speakerTestButton.disabled = true;
      }

      // 処理中のボタン無効化
      function setBtnunderProcessing() {
        h_startRecButton.disabled = true;
        h_stopRecButton.disabled = true;
        h_speakerSelect.disabled = true;
        h_speakerTestButton.disabled = true;
      }

      // 復帰時のボタン有効化
      function setBtnonRestart() {
        h_startRecButton.disabled = false;
        h_stopRecButton.disabled = true;
        h_speakerSelect.disabled = false;
        h_speakerTestButton.disabled = false;
      }


      document.addEventListener("DOMContentLoaded", () => {
        const socket = io();

        // SpeakerIDリストを取得
        fetch("/speaker_ids")
          .then((response) => response.json())
          .then((data) => {
            h_speakerSelect.innerHTML = data.join("");
          });

        // 音声認識の結果を受信
        socket.on("SpeechRecognition", (data) => {
          const markdownText = data.text;
          const htmlContent = marked.parse(markdownText);
          h_chatlog.innerHTML += `<div class="user">${htmlContent}</div>`;
        });

        // AIの応答を受信したときの処理
        socket.on("ai_response", (data) => {
          const markdownText = data.ai_response;
          const htmlContent = marked.parse(markdownText);
          h_chatlog.innerHTML += `<div class="assistant">${htmlContent}</div>`;
        });

        // AIの応答ストリームを受信したときの処理
        let currentDiv="";
        socket.on("ai_stream", (data) => {
          if (data.ai_stream.includes("---Start---")) { 
            // 最初はdivを作成
            h_chatlog.innerHTML += `<div class="assistant"></div>`;
            const assistantDivs = h_chatlog.getElementsByClassName("assistant");
            currentDiv = assistantDivs[assistantDivs.length - 1];//作ったdivを取得
            return;
          }
          else if (data.ai_stream.includes("---End---") ){ 
            // 終了時は改めて中身をマークダウンで書き直す．
            currentDiv.innerHTML= marked.parse(currentDiv.innerHTML);
            currentDiv = ""; //初期化
            return;
          }
          else{
            // 途中の場合はnakedなテキストを追加
            currentDiv.innerHTML += data.ai_stream;
          }
        });

        // 音声ファイルを再生する処理
        socket.on("play_audio", async(data) => {
            const audioBlob = new Blob([data.audio], { type: "audio/mp3" });
            const audioUrl = URL.createObjectURL(audioBlob);

            // キューに登録
            audioQueue.push(audioUrl);

            // 再生中でなければ再生
            if (!isPlaying) {
                playAudio();
            }
        });

        // Queueに登録された音声ファイルを再生する処理
        async function playAudio() {
            // 再生する音声ファイルがなければ終了
            if (audioQueue.length === 0) {
                isPlaying = false;
                return;
            }

            isPlaying = true;
            const audioUrl = audioQueue.shift();
            const audio = new Audio(audioUrl);
            audio.play();

            // 再生が終了したら次の音声ファイルを再生
            audio.onended = () => {
                playAudio();
            };
        }


        // Spaceキーが押されたときにstartRecordingボタンをクリック
        document.addEventListener("keydown", (event) => {
          if (h_startRecButton.disabled) {
            console.log("処理中のため入力はできません");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            h_startRecButton.click();
          }
        });

        // Spaceキーから指が離されたときにstopRecordingボタンをクリック
        document.addEventListener("keyup", (event) => {
          if (h_stopRecButton.disabled) {
            console.log("不正な録音停止操作です");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            h_stopRecButton.click();
          }
        });

        //Speakerの音声確認テスト
        h_speakerTestButton.addEventListener("click", () => {
          const speaker = h_speakerSelect.value;
          fetch("/speaker_test", {
            method: "POST",
            headers: {
              "Content-Type": "application/json",
            },
            body: JSON.stringify({ speaker }),
          })
            .then((response) => response.json())
            .then((data) => {
              console.log(data);
            });
        });

        // 録音開始ボタンがクリックされたときの処理
        h_startRecButton.addEventListener("click", () => {
          audioContext = new AudioContext();
          const source = audioContext.createMediaStreamSource(window.stream);
          recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
          recorder.record();

          // ボタンを無効化
          setBtnonStart();
        });

        // 録音停止ボタンがクリックされたときの処理
        h_stopRecButton.addEventListener("click", () => {
          // ボタンを無効化
          setBtnunderProcessing();

          // 録音を停止
          recorder.stop();

          // 録音した音声をファイルに保存して送信
          recorder.exportWAV((blob) => {
            audioBlob = blob;
            if (!audioBlob) {
              console.error("No audio to upload");
              return;
            }

            const formData = new FormData();
            formData.append("file", audioBlob, "recorded_audio.wav");

            const speaker = h_speakerSelect.value;
            formData.append("speaker", speaker);

            const method = document.querySelector('input[name="Method"]:checked').value;

            fetch(method, {
              method: "POST",
              body: formData,
            })
              .then((response) => response.json())
              .then((data) => {
                console.log(data);
                // ボタン状態の初期化
                setBtnonRestart();
              })
              .catch((error) => {
                console.error("Upload failed:");
                // ボタン状態の初期化
                setBtnonRestart();
              });
          });
        });

        // ボタン状態の初期化
        setBtnonRestart();
      });

      // ページを離れるときにストリームを停止
      window.addEventListener("beforeunload", () => {
        if (window.stream) {
          window.stream.getTracks().forEach((track) => {
            track.stop();
          });
        }
      });

    </script>
  </body>
</html>


なんとかできた．

Chunkの文章への再構築と，一方で文末かどうかの区切りがめちゃくちゃ面倒やった（苦笑）

現状では，音声生成のスピードがそこまでではないので，そこまでキュニー蓄積させるということもないが，もしもっと合成のスピードが上がれば，先に文章がバーっと表示されていく，ということが起こり得ると思う．
まあ，それも悪くはないが・・・


## その１５　音声とセンテンス表示の同期をとる．
チャレンジしてみるか

In [ ]:
%%writefile voicechatapp15.py

from flask import Flask, request, Response, jsonify, send_from_directory, send_file, stream_with_context
from flask_cors import CORS
from flask_socketio import SocketIO, emit
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
import requests
import time
import logging
import re
from pydub import AudioSegment
from io import BytesIO

#　環境変数の読み込み
load_dotenv()

# VoiceVox APIのエンドポイント
VOICEVOX_API_URL = "http://localhost:50021"


# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)

#--------------------------------------------------
# Flaskのエンドポイントの作成
#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index15.html")

# VoiceVoxのSpeakerIDリストを取得するエンドポイント
@app.route("/speaker_ids")
def get_speaker_ids():
    url = f"{VOICEVOX_API_URL}/speakers"  # VOICEVOX APIのエンドポイント
    try:
        response = requests.get(url)
    except Exception as e:
        logging.error(f"Error: {e}")
        return jsonify([])

    voicevox_speakers = []
    if response.status_code == 200:
        speakers = response.json()
        for speaker in speakers:
            name = speaker['name']
            style_names = [style['name'] for style in speaker['styles']]
            style_ids = [style['id'] for style in speaker['styles']]
            for style_id, style_name in zip(style_ids, style_names):
                voicevox_speakers.append(f"<option value={style_id}>Speaker: {name}, {style_name} </option>")
        logging.info("speaker_ids を取得しました。")
        return jsonify(voicevox_speakers)
    else:
        logging.error(f"Error: {response.status_code}")
        return jsonify([])    

# VoiceVoxの音声テストを行うエンドポイント
@app.route("/speaker_test" , methods=["POST"])
def speaker_test():
    speaker = request.json["speaker"]
    text = "こんにちは．初めまして．何かお手伝いできることはありますか？"
    synthesize_response = synthesize_voice(text, speaker)

    # 合成した音声をmp3化
    if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
    mp3_data  = BytesIO()
    audio.export(mp3_data , format="mp3")
    mp3_data .seek(0)  

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
    return jsonify({"info": "Speaker Test Process Succeeded"}), 200


# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav") #Uploadされたファイルを残すならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    
    # AIの応答を取得
    ai_response = get_ai_response(text)
    ## WebSocketを通じてクライアントに通知
    if ai_response:
        socketio.emit('ai_response', {'ai_response': ai_response}) 
    else:
        return jsonify({"error": "Failed to get AI response"}), 400
    
    # AIの応答から音声合成
    speaker = request.form["speaker"]
    synthesize_response = synthesize_voice(ai_response, speaker)


    # 合成した音声をmp3化
    if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
    mp3_data  = BytesIO()
    audio.export(mp3_data , format="mp3")
    mp3_data .seek(0)  

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})

    return jsonify({"info": "Uploard Process Succeeded"}), 200



# streaming処理するエンドポイント
@app.route("/streaming", methods=["POST"])
def streaming():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", "input.wav") #Uploadされたファイルを残さないならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    

    # AIの応答を句単位でストリームするとともに．句単位で音声合成もしていく
    speaker = request.form["speaker"]
    socketio.emit('ai_stream', {'sentens': "---Start---"}) # 開始を通知
    for sentence in generate_ai_response(text):
        ## WebSocketを通じてクライアントに通知
        if sentence:
            # 音声合成
            synthesize_response=synthesize_voice(sentence, speaker)
            if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
            ## 合成した音声をmp3化
            audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
            mp3_data  = BytesIO()
            audio.export(mp3_data , format="mp3")
            mp3_data .seek(0)
            ## mp3データをWebSocketを通じてクライアントに通知 ここでうまくキューに入れて連続再生させたい
            socketio.emit('ai_stream', {'audio': mp3_data.getvalue(), 'sentens': sentence})
            # sentensの区切り文字が読点だったら，0.2秒の無音を入れる
            if sentence[-1] in ",，、":
                silent_audio = AudioSegment.silent(duration=10)
                mp3_data  = BytesIO()
                silent_audio.export(mp3_data , format="mp3")
                mp3_data .seek(0)
            # sentensの区切り文字が読点でなかったら，0.5秒の無音を入れる
            else:
                silent_audio = AudioSegment.silent(duration=500)
                mp3_data  = BytesIO()
                silent_audio.export(mp3_data , format="mp3")
                mp3_data .seek(0)
            # 無音を送信
            socketio.emit('ai_stream', {'audio': mp3_data.getvalue(), 'sentens': "---silent---"})
        else:
            return jsonify({"error": "Failed to get AI response"}), 400
    socketio.emit('ai_stream', {'sentens': "---End---"}) # 終了を通知

    
    return jsonify({"info": "Process Succeeded"}), 200


#--------------------------------------------------
# Flaskの各エンドポイント内の処理関数
#--------------------------------------------------
# 音声認識を行う関数
def recognize_speech(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    return text

# OpenAIのAPIを呼び出してAIの応答を取得する関数
def get_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")
    return ai_response

# OpenAIのAPIを呼び出してAIの応答をストリームで生成する関数
def generate_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream = True
    )

    sentens = "" # 句を構成するためのバッファ　
    message = "" # プロンプトに含めるためにチャンクを結合させるためのためのバッファ
    for chunk in completion:
        # きちんとしたチャンクが帰ってきているかのチェック
        if "choices" in chunk.to_dict() and len(chunk.choices) > 0: #to_dict：辞書型に変えないと”choices”が見つからないようなので
            content  = chunk.choices[0].delta.content
            if content:
                message += content
                # 1文字ずつ取り出してチェックする
                for i in range(len(content)):
                    char = content[i]
                    sentens += char
                    if char in ",，、。．.？?！!\n": #今見ているのが区切り文字だった場合（読点も区切りに含める）
                        if i < len(content)-1: # i が最後の文字でないなら，次の文字をチェック
                            if content[i+1] not in ",，、。．.？?！!\n": #次の文字が区切り文字でないならyield
                                logging.debug(f"句: {sentens}")
                                yield sentens
                                sentens = ""
                            else: #もし次の文字が区切り文字なら，現時点の区切り文字はスルー
                                continue
                        else: #iが最後の文字の場合，現時点でyield
                            logging.debug(f"句: {sentens}")
                            yield sentens
                            sentens = ""
    # 最後の句を返す
    if sentens:
        yield sentens
    
    # message をmessagesに追加
    messages.append({"role": "assistant", "content": message})
    logging.info(f"AIの応答: {message}")



# VoiceVox APIで音声合成を行なう関数
def synthesize_voice(text, speaker):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'{VOICEVOX_API_URL}/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'{VOICEVOX_API_URL}/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        logging.info("音声データを生成しました。")
        return synthesis_response
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None


# テキストを句単位に区切る
def preprocess_text(text):
    # テキストの前処理
    text = re.sub(r"[。．.]", "。\n", text)
    text = re.sub(r"[？?]", "？\n", text)
    text = re.sub(r"[！!]", "！\n", text)
    return text


if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)




Writing voicechatapp15.py


In [ ]:
%%writefile static/index15.html
<html lang="ja">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>WAV録音アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css" />
</head>

<body>
    <h1>WAV録音アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording">録音停止</button>
    <input type="radio" id="streaming" name="Method" value="/upload" checked>まとめて再生(基本)</radio>
    <input type="radio" id="streaming" name="Method" value="/streaming">ストリーミング</radio>
    <select id="h_speakerSelect"></select>
    <button id="speakerTest">音声テスト</button>
    <div id="h_chatlog"></div>

    <script>
        navigator.mediaDevices
            .getUserMedia({ audio: true })
            .then((stream) => {
                window.stream = stream;
            })
            .catch((error) => {
                console.error("Error accessing the microphone: " + error);
            });

        // 音声処理用の変数
        let audioContext; // 音声処理用のコンテキスト
        let recorder;   // 録音用のオブジェクト
        let audioBlob;  // 録音した音声データ
        let audioQueue = [];    // 音声ファイルのキュー
        let sentensQueue = [];  // センテンスのキュー
        let isPlaying = false;  // 音声ファイル再生中かどうか
        let currentDiv = "";    // 現在のdiv要素


        // html要素取得
        const h_startRecButton = document.getElementById("startRecording");
        const h_stopRecButton = document.getElementById("stopRecording");
        const h_speakerSelect = document.getElementById("h_speakerSelect");
        const h_speakerTestButton = document.getElementById("speakerTest");
        const h_chatlog = document.getElementById("h_chatlog");

        // 録音開始時のボタンを無効化
        function setBtnonStart() {
            h_startRecButton.disabled = true;
            h_stopRecButton.disabled = false;
            h_speakerSelect.disabled = true;
            h_speakerTestButton.disabled = true;
        }

        // 処理中のボタン無効化
        function setBtnunderProcessing() {
            h_startRecButton.disabled = true;
            h_stopRecButton.disabled = true;
            h_speakerSelect.disabled = true;
            h_speakerTestButton.disabled = true;
        }

        // 復帰時のボタン有効化
        function setBtnonRestart() {
            h_startRecButton.disabled = false;
            h_stopRecButton.disabled = true;
            h_speakerSelect.disabled = false;
            h_speakerTestButton.disabled = false;
        }


        document.addEventListener("DOMContentLoaded", () => {
            const socket = io();

            // SpeakerIDリストを取得
            fetch("/speaker_ids")
                .then((response) => response.json())
                .then((data) => {
                    h_speakerSelect.innerHTML = data.join("");
                });

            // 音声認識の結果を受信
            socket.on("SpeechRecognition", (data) => {
                const markdownText = data.text;
                const htmlContent = marked.parse(markdownText);
                h_chatlog.innerHTML += `<div class="user">${htmlContent}</div>`;
            });

            // AIの応答を受信したときの処理
            socket.on("ai_response", (data) => {
                const markdownText = data.ai_response;
                const htmlContent = marked.parse(markdownText);
                h_chatlog.innerHTML += `<div class="assistant">${htmlContent}</div>`;
            });

            // 音声ファイルを再生する処理
            socket.on("play_audio", async (data) => {
                const audioBlob = new Blob([data.audio], { type: "audio/mp3" });
                const audioUrl = URL.createObjectURL(audioBlob);

                // キューに登録
                audioQueue.push(audioUrl);

                // 再生中でなければ再生
                if (!isPlaying) {
                    playAudio();
                }
                // const audio = new Audio(audioUrl);
                // audio.play();
            });

            // Queueに登録された音声ファイルを再生する処理
            async function playAudio() {
                // 再生する音声ファイルがなければ終了
                if (audioQueue.length === 0) {
                    isPlaying = false;
                    return;
                }

                isPlaying = true;
                const audioUrl = audioQueue.shift();
                const audio = new Audio(audioUrl);
                audio.play();

                // 再生が終了したら次の音声ファイルを再生
                audio.onended = () => {
                    playAudio();
                };
            }

            // AIの応答ストリームを受信したときの処理
            socket.on("ai_stream", (data) => {
                if(data.sentens){
                    if (data.sentens.includes("---Start---")) { 
                        // 最初はdivを作成
                        h_chatlog.innerHTML += `<div class="assistant"></div>`;
                        const assistantDivs = h_chatlog.getElementsByClassName("assistant");
                        currentDiv = assistantDivs[assistantDivs.length - 1];//作ったdivを取得
                        return;
                    }
                    else if (data.sentens.includes("---End---") ){ 
                        // 終了時はmarkedを適用
                        currentDiv.innerHTML= marked.parse(currentDiv.innerHTML);
                        currentDiv = ""; //初期化
                        return;
                    }
                    else{
                        // sentensをセンテンスキューに登録
                        sentensQueue.push(data.sentens);
                    }
                }

                if(data.audio){
                    // 音声ファイルをキューにと登録
                    const audioBlob = new Blob([data.audio], { type: "audio/mp3" });
                    const audioUrl = URL.createObjectURL(audioBlob);
                    audioQueue.push(audioUrl); // オーディオキューに登録


                    if (!isPlaying) {
                        playAudioWithSentens();
                    }
                }

            });

            // Queueに登録された音声ファイルを再生する処理
            async function playAudioWithSentens() {
                // 再生する音声ファイルがなければ終了
                if (audioQueue.length === 0) {
                    isPlaying = false;
                    return;
                }
                // 再生中フラグを立てる
                isPlaying = true;

                //SentensQueueからセンテンスを取り出して表示
                //ただし、---silent---が含まれている場合は表示しない
                const sentens = sentensQueue.shift();
                if (!sentens.includes("---silent---")){
                    currentDiv.innerHTML += sentens;
                }

                //AudioQueueから音声ファイルを取り出して再生
                const audioUrl = audioQueue.shift();
                const audio = new Audio(audioUrl);
                audio.play();

                // 再生が終了したら次の音声ファイルを再生
                audio.onended = () => {
                    playAudioWithSentens();
                };
            }

            // Spaceキーが押されたときにstartRecordingボタンをクリック
            document.addEventListener("keydown", (event) => {
                if (h_startRecButton.disabled) {
                    console.log("処理中のため入力はできません");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    h_startRecButton.click();
                }
            });

            // Spaceキーから指が離されたときにstopRecordingボタンをクリック
            document.addEventListener("keyup", (event) => {
                if (h_stopRecButton.disabled) {
                    console.log("不正な録音停止操作です");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    h_stopRecButton.click();
                }
            });

            //Speakerの音声確認テスト
            h_speakerTestButton.addEventListener("click", () => {
                const speaker = h_speakerSelect.value;
                fetch("/speaker_test", {
                    method: "POST",
                    headers: {
                        "Content-Type": "application/json",
                    },
                    body: JSON.stringify({ speaker }),
                })
                    .then((response) => response.json())
                    .then((data) => {
                        console.log(data);
                    });
            });

            // 録音開始ボタンがクリックされたときの処理
            h_startRecButton.addEventListener("click", () => {
                audioContext = new AudioContext();
                const source = audioContext.createMediaStreamSource(window.stream);
                recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
                recorder.record();

                // ボタンを無効化
                setBtnonStart();
            });

            // 録音停止ボタンがクリックされたときの処理
            h_stopRecButton.addEventListener("click", () => {
                // ボタンを無効化
                setBtnunderProcessing();

                // 録音を停止
                recorder.stop();

                // 録音した音声をファイルに保存して送信
                recorder.exportWAV((blob) => {
                    audioBlob = blob;
                    if (!audioBlob) {
                        console.error("No audio to upload");
                        return;
                    }

                    const formData = new FormData();
                    formData.append("file", audioBlob, "recorded_audio.wav");

                    const speaker = h_speakerSelect.value;
                    formData.append("speaker", speaker);

                    const method = document.querySelector('input[name="Method"]:checked').value;

                    fetch(method, {
                        method: "POST",
                        body: formData,
                    })
                        .then((response) => response.json())
                        .then((data) => {
                            console.log(data);
                            // ボタン状態の初期化
                            setBtnonRestart();
                        })
                        .catch((error) => {
                            console.error("Upload failed:");
                            // ボタン状態の初期化
                            setBtnonRestart();
                        });
                });
            });

            // ボタン状態の初期化
            setBtnonRestart();
        });

        // ページを離れるときにストリームを停止
        window.addEventListener("beforeunload", () => {
            if (window.stream) {
                window.stream.getTracks().forEach((track) => {
                    track.stop();
                });
            }
        });

    </script>
</body>

</html>

Writing static/index15.html


昨日，最後の一文が音声が流れたけど，センテンスが表示されなかった原因が多分分かった！
原因は，現在のコードだと，たとえセンテンスQueにセンテンスが残っていたとしても，ソケット通信でEndが送られるとHTMLを閉じてしまうコードになっているから．
対策として，
StartとEnd以外はかならずAudioとSentensがセットで送られるから，Endが発信されたということは，必要なAudioもSentensも全てQueに入っている．なので，EndもセンテンスQueに入れておいて，オーディオQueの長さが0になって時点で，センテンスQueを吐き出させればよい（通常はオーディオQueとSentensQueはセットで動くのでEndがQueに入った時だけこれが発動するはず！！

ということで，改めて以下の通り．

In [ ]:
%%writefile static/index15.html

<html lang="ja">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>WAV録音アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css" />
</head>

<body>
    <h1>WAV録音アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording">録音停止</button>
    <input type="radio" id="streaming" name="Method" value="/upload" checked>まとめて再生(基本)</radio>
    <input type="radio" id="streaming" name="Method" value="/streaming">ストリーミング</radio>
    <select id="h_speakerSelect"></select>
    <button id="speakerTest">音声テスト</button>
    <div id="h_chatlog"></div>

    <script>
        navigator.mediaDevices
            .getUserMedia({ audio: true })
            .then((stream) => {
                window.stream = stream;
            })
            .catch((error) => {
                console.error("Error accessing the microphone: " + error);
            });

        // 音声処理用の変数
        let audioContext; // 音声処理用のコンテキスト
        let recorder;   // 録音用のオブジェクト
        let audioBlob;  // 録音した音声データ
        let audioQueue = [];    // 音声ファイルのキュー
        let sentensQueue = [];  // センテンスのキュー
        let isPlaying = false;  // 音声ファイル再生中かどうか
        let currentDiv = "";    // 現在のdiv要素


        // html要素取得
        const h_startRecButton = document.getElementById("startRecording");
        const h_stopRecButton = document.getElementById("stopRecording");
        const h_speakerSelect = document.getElementById("h_speakerSelect");
        const h_speakerTestButton = document.getElementById("speakerTest");
        const h_chatlog = document.getElementById("h_chatlog");

        // 録音開始時のボタンを無効化
        function setBtnonStart() {
            h_startRecButton.disabled = true;
            h_stopRecButton.disabled = false;
            h_speakerSelect.disabled = true;
            h_speakerTestButton.disabled = true;
        }

        // 処理中のボタン無効化
        function setBtnunderProcessing() {
            h_startRecButton.disabled = true;
            h_stopRecButton.disabled = true;
            h_speakerSelect.disabled = true;
            h_speakerTestButton.disabled = true;
        }

        // 復帰時のボタン有効化
        function setBtnonRestart() {
            h_startRecButton.disabled = false;
            h_stopRecButton.disabled = true;
            h_speakerSelect.disabled = false;
            h_speakerTestButton.disabled = false;
        }


        document.addEventListener("DOMContentLoaded", () => {
            const socket = io();

            // SpeakerIDリストを取得
            fetch("/speaker_ids")
                .then((response) => response.json())
                .then((data) => {
                    h_speakerSelect.innerHTML = data.join("");
                });

            // 音声認識の結果を受信
            socket.on("SpeechRecognition", (data) => {
                const markdownText = data.text;
                const htmlContent = marked.parse(markdownText);
                h_chatlog.innerHTML += `<div class="user">${htmlContent}</div>`;
            });

            // AIの応答を受信したときの処理
            socket.on("ai_response", (data) => {
                const markdownText = data.ai_response;
                const htmlContent = marked.parse(markdownText);
                h_chatlog.innerHTML += `<div class="assistant">${htmlContent}</div>`;
            });

            // 音声ファイルを再生する処理
            socket.on("play_audio", async (data) => {
                const audioBlob = new Blob([data.audio], { type: "audio/mp3" });
                const audioUrl = URL.createObjectURL(audioBlob);

                // キューに登録
                audioQueue.push(audioUrl);

                // 再生中でなければ再生
                if (!isPlaying) {
                    playAudio();
                }
                // const audio = new Audio(audioUrl);
                // audio.play();
            });

            // Queueに登録された音声ファイルを再生する処理
            async function playAudio() {
                // 再生する音声ファイルがなければ終了
                if (audioQueue.length === 0) {
                    isPlaying = false;
                    return;
                }

                isPlaying = true;
                const audioUrl = audioQueue.shift();
                const audio = new Audio(audioUrl);
                audio.play();

                // 再生が終了したら次の音声ファイルを再生
                audio.onended = () => {
                    playAudio();
                };
            }

            // AIの応答ストリームを受信したときの処理
            socket.on("ai_stream", (data) => {
                if(data.sentens){
                    if (data.sentens.includes("---Start---")) { 
                        // 最初はdivを作成
                        h_chatlog.innerHTML += `<div class="assistant"></div>`;
                        const assistantDivs = h_chatlog.getElementsByClassName("assistant");
                        currentDiv = assistantDivs[assistantDivs.length - 1];//作ったdivを取得
                        return;
                    }
                    // else if (data.sentens.includes("---End---") ){ 
                    //     // 終了時はmarkedを適用
                    //     currentDiv.innerHTML= marked.parse(currentDiv.innerHTML);
                    //     currentDiv = ""; //初期化
                    //     return;
                    // }
                    else{
                        // sentensをセンテンスキューに登録
                        sentensQueue.push(data.sentens);
                    }
                }

                if(data.audio){
                    // 音声ファイルをキューにと登録
                    const audioBlob = new Blob([data.audio], { type: "audio/mp3" });
                    const audioUrl = URL.createObjectURL(audioBlob);
                    audioQueue.push(audioUrl); // オーディオキューに登録

                    if (!isPlaying) {
                        playAudioWithSentens();
                    }
                }
            });

            // Queueに登録された音声ファイルを再生する処理
            async function playAudioWithSentens() {
                // 再生する音声ファイルがなければ終了
                if (audioQueue.length === 0) {
                    isPlaying = false;
                    //もしセンテンスQueにデータがあれば全部吐き出す
                    while (sentensQueue.length)  {
                        const sentens = sentensQueue.shift();
                        if (sentens.includes("---End---")){
                            currentDiv.innerHTML= marked.parse(currentDiv.innerHTML);
                            currentDiv = ""; //初期化
                        }else{
                            currentDiv.innerHTML += sentens;
                        }
                    }
                    return;
                }
                // 再生中フラグを立てる
                isPlaying = true;

                //SentensQueueからセンテンスを取り出して表示
                //ただし、---silent---が含まれている場合は表示しない
                const sentens = sentensQueue.shift();
                if (!sentens.includes("---silent---")){
                    currentDiv.innerHTML += sentens;
                }

                //AudioQueueから音声ファイルを取り出して再生
                const audioUrl = audioQueue.shift();
                const audio = new Audio(audioUrl);
                audio.play();

                // 再生が終了したら次の音声ファイルを再生
                audio.onended = () => {
                    playAudioWithSentens();
                };
            }

            // Spaceキーが押されたときにstartRecordingボタンをクリック
            document.addEventListener("keydown", (event) => {
                if (h_startRecButton.disabled) {
                    console.log("処理中のため入力はできません");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    h_startRecButton.click();
                }
            });

            // Spaceキーから指が離されたときにstopRecordingボタンをクリック
            document.addEventListener("keyup", (event) => {
                if (h_stopRecButton.disabled) {
                    console.log("不正な録音停止操作です");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    h_stopRecButton.click();
                }
            });

            //Speakerの音声確認テスト
            h_speakerTestButton.addEventListener("click", () => {
                const speaker = h_speakerSelect.value;
                fetch("/speaker_test", {
                    method: "POST",
                    headers: {
                        "Content-Type": "application/json",
                    },
                    body: JSON.stringify({ speaker }),
                })
                    .then((response) => response.json())
                    .then((data) => {
                        console.log(data);
                    });
            });

            // 録音開始ボタンがクリックされたときの処理
            h_startRecButton.addEventListener("click", () => {
                audioContext = new AudioContext();
                const source = audioContext.createMediaStreamSource(window.stream);
                recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
                recorder.record();

                // ボタンを無効化
                setBtnonStart();
            });

            // 録音停止ボタンがクリックされたときの処理
            h_stopRecButton.addEventListener("click", () => {
                // ボタンを無効化
                setBtnunderProcessing();

                // 録音を停止
                recorder.stop();

                // 録音した音声をファイルに保存して送信
                recorder.exportWAV((blob) => {
                    audioBlob = blob;
                    if (!audioBlob) {
                        console.error("No audio to upload");
                        return;
                    }

                    const formData = new FormData();
                    formData.append("file", audioBlob, "recorded_audio.wav");

                    const speaker = h_speakerSelect.value;
                    formData.append("speaker", speaker);

                    const method = document.querySelector('input[name="Method"]:checked').value;

                    fetch(method, {
                        method: "POST",
                        body: formData,
                    })
                        .then((response) => response.json())
                        .then((data) => {
                            console.log(data);
                            // ボタン状態の初期化
                            setBtnonRestart();
                        })
                        .catch((error) => {
                            console.error("Upload failed:");
                            // ボタン状態の初期化
                            setBtnonRestart();
                        });
                });
            });

            // ボタン状態の初期化
            setBtnonRestart();
        });

        // ページを離れるときにストリームを停止
        window.addEventListener("beforeunload", () => {
            if (window.stream) {
                window.stream.getTracks().forEach((track) => {
                    track.stop();
                });
            }
        });

    </script>
</body>

</html>

## その16　他のTTSに対応する 

とりあえずGoogle Cloud TTSを使ってみるか.
ついでにvoicevoxについてmp3出力を端から出来るようにしておく．

### とりあえずGoogle Clout TTSを使えるようにする

In [1]:
import requests
import json
import io
from pydub import AudioSegment
from pydub.playback import play
import base64
from dotenv import load_dotenv
import os

# 環境変数の読み込み
load_dotenv()

# API_KEYの取得
API_KEY = os.environ.get("GOOGLE_TTS_API_KEY")

# APIエンドポイント
url = f"https://texttospeech.googleapis.com/v1/text:synthesize?key={API_KEY}"

# 音声合成のリクエストデータ
data = {
    "input": {"text": "Hello Nice to meet you. How can I help you? I am a helpful assistant. Please tell me your request. I will do my best to help you."},
    "voice": {
        "languageCode": "en-US",
        "name": "en-US-Journey-F",  
        #"ssmlGender": "FEMALE"
    },
    "audioConfig": {
        "audioEncoding": "MP3"
    }
}

# リクエスト送信
response = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps(data))

# 結果を取得
if response.status_code == 200:
    # Base64エンコードされた音声データをデコード
    audio_content = json.loads(response.text)["audioContent"]
    audio_data = base64.b64decode(audio_content)
    
    # バイナリデータを pydub の AudioSegment に変換
    audio = AudioSegment.from_file(io.BytesIO(audio_data), format="mp3")

    # Python 上で直接再生
    play(audio)

else:
    print("エラー:", response.text)



Input #0, wav, from '/var/folders/vb/63y84xfs4g3cw7_09_s2_yd40000gn/T/tmp_vhok6s3.wav':
  Duration: 00:00:09.02, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s


ふむ．PYTHONライブラリを使うより，API KEYをつかうのが簡単だよね（苦笑）

ということでGoogle TTSを組み込んだバージョン．
VoiceVoxよりレスポンスはやいからスムーズにきこえるね．

In [ ]:
%%writefile voicechatapp16.py

from flask import Flask, request, Response, jsonify, send_from_directory, send_file, stream_with_context
from flask_cors import CORS
from flask_socketio import SocketIO, emit
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
import requests
import time
import logging
import json
from pydub import AudioSegment
from io import BytesIO
import base64


#　環境変数の読み込み
load_dotenv()

# VoiceVox APIのエンドポイント
VOICEVOX_API_URL = "http://localhost:50021"


# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

# 区切り文字の設定．AI出力をストリームで受け取るときに句切りをどの文字で行なうかの指定
# この文字が来たら，その前までを一つの句として扱う
SegmentingChars=",，、。．.？?！!\n"

#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)

#--------------------------------------------------
# Flaskのエンドポイントの作成
#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index15.html")

# VoiceVoxのSpeakerIDリストを取得するエンドポイント
@app.route("/speaker_ids")
def get_speaker_ids():
    url = f"{VOICEVOX_API_URL}/speakers"  # VOICEVOX APIのエンドポイント
    try:
        response = requests.get(url)
    except Exception as e:
        logging.error(f"Error: {e}")
        return jsonify([])

    voicevox_speakers = []
    if response.status_code == 200:
        speakers = response.json()
        for speaker in speakers:
            name = speaker['name']
            style_names = [style['name'] for style in speaker['styles']]
            style_ids = [style['id'] for style in speaker['styles']]
            for style_id, style_name in zip(style_ids, style_names):
                voicevox_speakers.append(f"<option value={style_id}>Speaker: {name}, {style_name} </option>")
        logging.info("speaker_ids を取得しました。")
        return jsonify(voicevox_speakers)
    else:
        logging.error(f"Error: {response.status_code}")
        return jsonify([])    

# VoiceVoxの音声テストを行うエンドポイント
@app.route("/speaker_test" , methods=["POST"])
def speaker_test():
    speaker = request.json["speaker"]
    text = "こんにちは．初めまして．何かお手伝いできることはありますか？"
    mp3_data = synthesize_voice_mp3(text, speaker)
    if mp3_data is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
    return jsonify({"info": "Speaker Test Process Succeeded"}), 200


# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav") #Uploadされたファイルを残すならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    
    # AIの応答を取得
    ai_response = get_ai_response(text)
    ## WebSocketを通じてクライアントに通知
    if ai_response:
        socketio.emit('ai_response', {'ai_response': ai_response}) 
    else:
        return jsonify({"error": "Failed to get AI response"}), 400
    
    # AIの応答から音声合成してmp3で返す
    speaker = request.form["speaker"]
    mp3_data = synthesize_voice_mp3(ai_response, speaker)
    if mp3_data is None: return jsonify({"error": "Failed to synthesize voice"}), 400

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})

    return jsonify({"info": "Uploard Process Succeeded"}), 200



# streaming処理するエンドポイント
@app.route("/streaming", methods=["POST"])
def streaming():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", "input.wav") #Uploadされたファイルを残さないならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    

    # AIの応答を句単位でストリームするとともに．句単位で音声合成もしていく
    speaker = request.form["speaker"]
    socketio.emit('ai_stream', {'sentens': "---Start---"}) # 開始を通知
    for sentence in generate_ai_response(text):
        ## WebSocketを通じてクライアントに通知
        if sentence:
            #　音声合成（mp3出力）
            #mp3_data = synthesize_voice_mp3(sentence, speaker) # VoiceVox APIを使う場合
            mp3_data = synthesize_voice_google(sentence) # Google Cloud TTS APIを使う場合
            if mp3_data is None: return jsonify({"error": "Failed to synthesize voice"}), 400
            ## mp3データをWebSocketを通じてクライアントに通知 ここでうまくキューに入れて連続再生させたい
            socketio.emit('ai_stream', {'audio': mp3_data.getvalue(), 'sentens': sentence})
            
            # sentensの区切り文字が読点だったら，0.2秒の無音を入れる
            if sentence[-1] in ",，、":
                silent_audio = AudioSegment.silent(duration=10)
                mp3_data  = BytesIO()
                silent_audio.export(mp3_data , format="mp3")
                mp3_data .seek(0)
            # sentensの区切り文字が読点でなかったら，0.5秒の無音を入れる
            else:
                silent_audio = AudioSegment.silent(duration=500)
                mp3_data  = BytesIO()
                silent_audio.export(mp3_data , format="mp3")
                mp3_data .seek(0)
            # 無音を送信
            socketio.emit('ai_stream', {'audio': mp3_data.getvalue(), 'sentens': "---silent---"})
        else:
            return jsonify({"error": "Failed to get AI response"}), 400
    socketio.emit('ai_stream', {'sentens': "---End---"}) # 終了を通知

    
    return jsonify({"info": "Process Succeeded"}), 200


#--------------------------------------------------
# Flaskの各エンドポイント内の処理関数
#--------------------------------------------------
# 音声認識を行う関数
def recognize_speech(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    return text

# OpenAIのAPIを呼び出してAIの応答を取得する関数
def get_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")
    return ai_response

# OpenAIのAPIを呼び出してAIの応答をストリームで生成する関数
def generate_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream = True
    )

    sentens = "" # 句を構成するためのバッファ　
    message = "" # プロンプトに含めるためにチャンクを結合させるためのためのバッファ
    for chunk in completion:
        # きちんとしたチャンクが帰ってきているかのチェック
        if "choices" in chunk.to_dict() and len(chunk.choices) > 0: #to_dict：辞書型に変えないと”choices”が見つからないようなので
            content  = chunk.choices[0].delta.content
            if content:
                message += content
                # 1文字ずつ取り出してチェックする
                for i in range(len(content)):
                    char = content[i]
                    sentens += char
                    if char in SegmentingChars: #今見ているのが区切り文字だった場合（読点も区切りに含める）
                        if i < len(content)-1: # i が最後の文字でないなら，次の文字をチェック
                            if content[i+1] not in SegmentingChars: #次の文字が区切り文字でないならyield
                                logging.debug(f"句: {sentens}")
                                yield sentens
                                sentens = ""
                            else: #もし次の文字が区切り文字なら，現時点の区切り文字はスルー
                                continue
                        else: #iが最後の文字の場合，現時点でyield
                            logging.debug(f"句: {sentens}")
                            yield sentens
                            sentens = ""
    # 最後の句を返す
    if sentens:
        yield sentens
    
    # message をmessagesに追加
    messages.append({"role": "assistant", "content": message})
    logging.info(f"AIの応答: {message}")



# VoiceVox APIで音声合成を行なう関数
def synthesize_voice(text, speaker):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'{VOICEVOX_API_URL}/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'{VOICEVOX_API_URL}/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        logging.info("音声データを生成しました。")
        return synthesis_response
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None


# voicevox apiで音声合成を行う関数（mp3出力）
def synthesize_voice_mp3(text, speaker):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'{VOICEVOX_API_URL}/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'{VOICEVOX_API_URL}/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        logging.info("音声データを生成しました。")
        audio = AudioSegment.from_file(BytesIO(synthesis_response.content), format="wav")
        mp3_data  = BytesIO()
        audio.export(mp3_data , format="mp3")
        mp3_data .seek(0)  
        return mp3_data
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None

# Google Clout TTS APIで音声合成を行う関数
def synthesize_voice_google(text):
    # APIキーの取得
    API_KEY = os.getenv("GOOGLE_TTS_API_KEY")

    # APIエンドポイント
    url = f"https://texttospeech.googleapis.com/v1/text:synthesize?key={API_KEY}"

    # 音声合成のリクエストデータ
    data = {
        "input": {"text": text},
        "voice": {
            "languageCode": "ja-JP",
            "name": "ja-JP-Wavenet-D",  # 男性の自然な声
            "ssmlGender": "MALE"
        },
        "audioConfig": {
            "audioEncoding": "MP3"
        }
    }

    # リクエスト送信
    response = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps(data))

    # 結果を取得
    if response.status_code == 200:
        # Base64エンコードされた音声データをデコード
        audio_content = json.loads(response.text)["audioContent"]
        audio_data = base64.b64decode(audio_content)
        
        # バイナリデータを pydub の AudioSegment に変換
        mp3_data  =BytesIO(audio_data)
        mp3_data .seek(0)  
        return mp3_data
    else:
        logging.error(f"Error in synthesis: {response.text}")
        return None
    
if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)


Writing voicechatapp16.py


HTMLも弄っておいて，Googleの場合とVoiceVoxの場合の選択が出来るようにしておくか．

### HTMLで色々と触れるようにする

https://cloud.google.com/text-to-speech/docs/voices?hl=ja

https://cloud.google.com/text-to-speech/docs/voice-types?hl=ja

声の種類が思いのほか多い😂
日本語はともかく，英語はめちゃくちゃ多い😂

In [ ]:
%%writefile static/index16.html

<html lang="ja">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>WAV録音アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css" />
</head>

<body>
    <h1>WAV録音アップロード</h1>
    <button type="button" id="startRecording">録音開始</button>
    <button type="button" id="stopRecording">録音停止</button>
    <form id = "myForm">
        <div id ="divProcessType">
            <input type="radio" id="onetime" name="Method" value="/upload" checked>まとめて再生(基本)</radio>
            <input type="radio" id="streaming" name="Method" value="/streaming">ストリーミング</radio>
        </div>
        <div id ="divTTSselect">
            <input type="radio" id="radioVoicevoxTTS" name="TTS" value="VoiceVox" checked>VoiceVox</radio>
            <input type="radio" id="radioGoogleTTS" name="TTS" value="Google">Google TTS</radio>
        </div>
        <div id ="divVoiceVoxSpeaker">
            <select id="speakerSelect" name="speaker"></select>
        </div>
        <div id ="divGoogleSpeaker" hidden>
            <p>言語
                <input type="radio"  id = "langCode_jp" name="languageCode" value="ja-JP" checked>日本語</radio>
                <input type="radio"  id = "langCode_en" name="languageCode" value="en-US">英語</radio>
            </p>
            <p id="JPvoiceSelect">日本語の声質
                <select id="JPvoicetype" name="JPvoicetype">
                    <option value="ja-JP-Neural2-B">ニューラル・女性</option>
                    <option value="ja-JP-Neural2-C">ニューラル・男性1</option>
                    <option value="ja-JP-Neural2-D">ニューラル男性2</option>
                    <option value="ja-JP-Wavenet-A">ウェーブネット・女性1</option>
                    <option value="ja-JP-Wavenet-B">ウェーブネット・女性2</option>
                    <option value="ja-JP-Wavenet-C">ウェーブネット・男性1</option>
                    <option value="ja-JP-Wavenet-D">ウェーブネット・男性2</option>
                </select>
            </p>
            <p id="ENvoiceSelect" hidden>英語の声質
                <select id="ENvoicetype" name="ENvoicetype">
                    <option value="en-US-Journey-F">ジャーニー/女性1</option>
                    <option value="en-US-Journey-O">ジャーニー/女性2</option>
                    <option value="en-US-Journey-D">ジャーニー/男性1</option>
                    <option value="en-US-Wavenet-C">ウェーブネット/女性1</option>
                    <option value="en-US-Wavenet-E">ウェーブネット/女性2</option>
                    <option value="en-US-Wavenet-F">ウェーブネット/女性3</option>
                    <option value="en-US-Wavenet-G">ウェーブネット/女性4</option>
                    <option value="en-US-Wavenet-H">ウェーブネット/女性5</option>
                    <option value="en-US-Wavenet-A">ウェーブネット/男性1</option>
                    <option value="en-US-Wavenet-B">ウェーブネット/男性2</option>
                    <option value="en-US-Wavenet-D">ウェーブネット/男性3</option>
                    <option value="en-US-Wavenet-I">ウェーブネット/男性4</option>
                    <option value="en-US-Wavenet-J">ウェーブネット/男性5</option>
                </select>
            </p>
        </div>
        <button type="button" id="speakerTest">音声テスト</button>
    </form>
    <div id="chatlog"></div>

    <script>
        navigator.mediaDevices
            .getUserMedia({ audio: true })
            .then((stream) => {
                window.stream = stream;
            })
            .catch((error) => {
                console.error("Error accessing the microphone: " + error);
            });

        // 音声処理用の変数
        let audioContext; // 音声処理用のコンテキスト
        let recorder;   // 録音用のオブジェクト
        let audioBlob;  // 録音した音声データ
        let audioQueue = [];    // 音声ファイルのキュー
        let sentensQueue = [];  // センテンスのキュー
        let isPlaying = false;  // 音声ファイル再生中かどうか
        let currentDiv = "";    // 現在のdiv要素


        // html要素取得
        const h_startRecButton = document.getElementById("startRecording");
        const h_stopRecButton = document.getElementById("stopRecording");
        const h_speakerSelect = document.getElementById("speakerSelect");
        const h_speakerTestButton = document.getElementById("speakerTest");
        const h_chatlog = document.getElementById("chatlog");
        const h_languageCode = document.querySelector('input[name="languageCode"]:checked');
        const h_jpname = document.getElementById("jpname");
        const h_enname = document.getElementById("enname");
        const h_radioVoicevoxTTS = document.getElementById("radioVoicevoxTTS");
        const h_radioGoogleTTS = document.getElementById("radioGoogleTTS");   
        const h_TTS = document.querySelector('input[name="TTS"]:checked');        

        // 録音開始時のボタンを無効化
        function setBtnonStart() {
            h_startRecButton.disabled = true;
            h_stopRecButton.disabled = false;
            h_speakerSelect.disabled = true;
            h_speakerTestButton.disabled = true;
        }

        // 処理中のボタン無効化
        function setBtnunderProcessing() {
            h_startRecButton.disabled = true;
            h_stopRecButton.disabled = true;
            h_speakerSelect.disabled = true;
            h_speakerTestButton.disabled = true;
        }

        // 復帰時のボタン有効化
        function setBtnonRestart() {
            h_startRecButton.disabled = false;
            h_stopRecButton.disabled = true;
            h_speakerSelect.disabled = false;
            h_speakerTestButton.disabled = false;
        }

        // formsの値を取得してJSON形式で返す
        function getFormValues(){
            const data = new FormData(document.getElementById("myForm"));
            const obj = {};
            data.forEach((value, key) => {
                obj[key] = value;
            });
            console.log(obj);
            return obj;
        }
     

        document.addEventListener("DOMContentLoaded", () => {
            // Socket.IO サーバーに接続
            const socket = io();

            // VoiceVoxの話者リストを取得
            fetch("/speaker_ids")
                .then((response) => response.json())
                .then((data) => {
                    h_speakerSelect.innerHTML = data.join("");
                });


/****** socket.ioの処理 *****/
            // 音声認識の結果を受信
            socket.on("SpeechRecognition", (data) => {
                const markdownText = data.text;
                const htmlContent = marked.parse(markdownText);
                h_chatlog.innerHTML += `<div class="user">${htmlContent}</div>`;
            });

            // AIの応答を受信したときの処理
            socket.on("ai_response", (data) => {
                const markdownText = data.ai_response;
                const htmlContent = marked.parse(markdownText);
                h_chatlog.innerHTML += `<div class="assistant">${htmlContent}</div>`;
            });

            // 音声を再生する処理
            socket.on("play_audio", async (data) => {
                const audioBlob = new Blob([data.audio], { type: "audio/mp3" });
                const audioUrl = URL.createObjectURL(audioBlob);

                // キューに登録
                audioQueue.push(audioUrl);

                // 再生中でなければ再生
                if (!isPlaying) {
                    playAudio();
                }
                // const audio = new Audio(audioUrl);
                // audio.play();
            });

            // Queueに登録された音声ファイルを再生する処理
            async function playAudio() {
                // 再生する音声ファイルがなければ終了
                if (audioQueue.length === 0) {
                    isPlaying = false;
                    return;
                }

                isPlaying = true;
                const audioUrl = audioQueue.shift();
                const audio = new Audio(audioUrl);
                audio.play();

                // 再生が終了したら次の音声ファイルを再生
                audio.onended = () => {
                    playAudio();
                };
            }

            // AIの応答ストリームを受信したときの処理
            socket.on("ai_stream", (data) => {
                if(data.sentens){
                    if (data.sentens.includes("---Start---")) { 
                        // 最初はdivを作成
                        h_chatlog.innerHTML += `<div class="assistant"></div>`;
                        const assistantDivs = h_chatlog.getElementsByClassName("assistant");
                        currentDiv = assistantDivs[assistantDivs.length - 1];//作ったdivを取得
                        return;
                    }
                    // else if (data.sentens.includes("---End---") ){ 
                    //     // 終了時はmarkedを適用
                    //     currentDiv.innerHTML= marked.parse(currentDiv.innerHTML);
                    //     currentDiv = ""; //初期化
                    //     return;
                    // }
                    else{
                        // sentensをセンテンスキューに登録
                        sentensQueue.push(data.sentens);
                    }
                }

                if(data.audio){
                    // 音声ファイルをキューに登録
                    const audioBlob = new Blob([data.audio], { type: "audio/mp3" });
                    const audioUrl = URL.createObjectURL(audioBlob);
                    audioQueue.push(audioUrl); // オーディオキューに登録

                    if (!isPlaying) {
                        playAudioWithSentens();
                    }
                }
            });

            // Queueに登録された音声ファイルを再生する処理
            async function playAudioWithSentens() {
                // 再生する音声ファイルがなければ終了
                if (audioQueue.length === 0) {
                    isPlaying = false;
                    //もしセンテンスQueにデータがあれば全部吐き出す
                    while (sentensQueue.length)  {
                        const sentens = sentensQueue.shift();
                        if (sentens.includes("---End---")){
                            currentDiv.innerHTML= marked.parse(currentDiv.innerHTML);
                            currentDiv = ""; //初期化
                        }else{
                            currentDiv.innerHTML += sentens;
                        }
                    }
                    return;
                }
                // 再生中フラグを立てる
                isPlaying = true;

                //SentensQueueからセンテンスを取り出して表示
                //ただし---silent---が含まれている場合は表示しない
                const sentens = sentensQueue.shift();
                if (!sentens.includes("---silent---")){
                    currentDiv.innerHTML += sentens;
                }

                //AudioQueueから音声ファイルを取り出して再生
                const audioUrl = audioQueue.shift();
                const audio = new Audio(audioUrl);
                audio.play();

                // 再生が終了したら次の音声ファイルを再生
                audio.onended = () => {
                    playAudioWithSentens();
                };
            }


/***** Event listener *****/
            // TTSselectの選択による表示切り替え
            //// もしVoiceVoxが選択されていたら，divVoiceVoxSpeakerを表示し，divGoogleSpeakerを非表示にする
            h_radioVoicevoxTTS.addEventListener("click", () => {
                document.getElementById("divVoiceVoxSpeaker").hidden = false;
                document.getElementById("divGoogleSpeaker").hidden = true;
            });
            //// もしGoogleTTSが選択されていたら，divVoiceVoxSpeakerを非表示し，divGoogleSpeakerを表示する
            h_radioGoogleTTS.addEventListener("click", () => {
                document.getElementById("divVoiceVoxSpeaker").hidden = true;
                document.getElementById("divGoogleSpeaker").hidden = false;
            });

            // GoogleTTSの言語選択による表示切り替え
            //// もし日本語が選択されていたら，JPvoiceSelectを表示し，ENvoiceSelectを非表示にする
            document.getElementById("langCode_jp").addEventListener("click", () => {
                document.getElementById("JPvoiceSelect").hidden = false;
                document.getElementById("ENvoiceSelect").hidden = true;
            });
            //// もし英語が選択されていたら，JPvoiceSelectを非表示し，ENvoiceSelectを表示する   
            document.getElementById("langCode_en").addEventListener("click", () => {
                document.getElementById("JPvoiceSelect").hidden = true;
                document.getElementById("ENvoiceSelect").hidden = false;
            });

            // Spaceキーが押されたときにstartRecordingボタンをクリック
            document.addEventListener("keydown", (event) => {
                if (h_startRecButton.disabled) {
                    console.log("処理中のため入力はできません");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    h_startRecButton.click();
                }
            });

            // Spaceキーから指が離されたときにstopRecordingボタンをクリック
            document.addEventListener("keyup", (event) => {
                if (h_stopRecButton.disabled) {
                    console.log("不正な録音停止操作です");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    h_stopRecButton.click();
                }
            });

            //Speakerの音声確認テスト
            h_speakerTestButton.addEventListener("click", () => {
                const speaker = speakerSelect.value;
                //Formの値を取得
                const data = getFormValues();
                fetch("/speaker_test", {
                    method: "POST",
                    headers: {
                        "Content-Type": "application/json",
                    },
                    body: JSON.stringify(data),
                })
                    .then((response) => response.json())
                    .then((data) => {
                        console.log(data);
                    });
            });

            // 録音開始ボタンがクリックされたときの処理
            h_startRecButton.addEventListener("click", () => {
                audioContext = new AudioContext();
                const source = audioContext.createMediaStreamSource(window.stream);
                recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
                recorder.record();

                // ボタンを無効化
                setBtnonStart();
            });

            // 録音停止ボタンがクリックされたときの処理
            h_stopRecButton.addEventListener("click", () => {
                // ボタンを無効化
                setBtnunderProcessing();

                // 録音を停止
                recorder.stop();

                // 録音した音声をファイルに保存して送信
                recorder.exportWAV((blob) => {
                    audioBlob = blob;
                    if (!audioBlob) {
                        console.error("No audio to upload");
                        return;
                    }

                    const formData = new FormData();
                    formData.append("file", audioBlob, "recorded_audio.wav");

                    const speaker = h_speakerSelect.value;
                    formData.append("speaker", speaker);

                    const method = document.querySelector('input[name="Method"]:checked').value;

                    fetch(method, {
                        method: "POST",
                        body: formData,
                    })
                        .then((response) => response.json())
                        .then((data) => {
                            console.log(data);
                            // ボタン状態の初期化
                            setBtnonRestart();
                        })
                        .catch((error) => {
                            console.error("Upload failed:");
                            // ボタン状態の初期化
                            setBtnonRestart();
                        });
                });
            });

            // ボタン状態の初期化
            setBtnonRestart();
        });

        // ページを離れるときにストリームを停止
        window.addEventListener("beforeunload", () => {
            if (window.stream) {
                window.stream.getTracks().forEach((track) => {
                    track.stop();
                });
            }
        });

    </script>
</body>

</html>


Writing static/index16.html


In [ ]:
%%writefile voicechatapp16.py


from flask import Flask, request, Response, jsonify, send_from_directory, send_file, stream_with_context
from flask_cors import CORS
from flask_socketio import SocketIO, emit
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
import requests
import time
import logging
import json
from pydub import AudioSegment
from io import BytesIO
import base64


#　環境変数の読み込み
load_dotenv()

# VoiceVox APIのエンドポイント
VOICEVOX_API_URL = "http://localhost:50021"


# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

# 区切り文字の設定．AI出力をストリームで受け取るときに句切りをどの文字で行なうかの指定
# この文字が来たら，その前までを一つの句として扱う
SegmentingChars=",，、。．.？?！!\n"

#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)

#--------------------------------------------------
# Flaskのエンドポイントの作成
#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index16.html")

# VoiceVoxのSpeakerIDリストを取得するエンドポイント
@app.route("/speaker_ids")
def get_speaker_ids():
    url = f"{VOICEVOX_API_URL}/speakers"  # VOICEVOX APIのエンドポイント
    try:
        response = requests.get(url)
    except Exception as e:
        logging.error(f"Error: {e}")
        return jsonify([])

    voicevox_speakers = []
    if response.status_code == 200:
        speakers = response.json()
        for speaker in speakers:
            name = speaker['name']
            style_names = [style['name'] for style in speaker['styles']]
            style_ids = [style['id'] for style in speaker['styles']]
            for style_id, style_name in zip(style_ids, style_names):
                voicevox_speakers.append(f"<option value={style_id}>Speaker: {name}, {style_name} </option>")
        logging.info("speaker_ids を取得しました。")
        return jsonify(voicevox_speakers)
    else:
        logging.error(f"Error: {response.status_code}")
        return jsonify([])    

# 音声テストを行うエンドポイント
@app.route("/speaker_test" , methods=["POST"])
def speaker_test():
    TTS = request.json["TTS"]
    speaker = request.json["speaker"]
    languageCode = request.json["languageCode"]
    JPvoicetype = request.json["JPvoicetype"]
    ENvoicetype = request.json["ENvoicetype"]
    print(f"speaker_test: TTS={TTS}, speaker={speaker}, languageCode={languageCode}, JPvoicetype={JPvoicetype}, ENvoicetype={ENvoicetype}")

    if TTS == "VoiceVox":
        text = "こんにちは．初めまして．何かお手伝いできることはありますか？"
        mp3_data = synthesize_voicevox_mp3(text, speaker)
    elif TTS == "Google":
        # 日本語と英語で分岐
        if languageCode == "ja-JP":
            text = f"こんにちは．初めまして．何かお手伝いできることはありますか？"
            voicetype = JPvoicetype
        elif languageCode == "en-US":
            text = f"Hello. Nice to meet you. How can I help you?"
            voicetype = ENvoicetype
        else:
            return jsonify({"error": "Failed to synthesize voice_Test. Input languageCode is irregal"}), 400
        # Google Cloud TTS APIで音声合成
        mp3_data = synthesize_voice_google(text,languageCode, voicetype)
    else:
        return jsonify({"error": "Failed to synthesize voice_Test. Input TTS is irregal"}), 400
    if mp3_data is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
    return jsonify({"info": "Speaker Test Process Succeeded"}), 200


# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav") #Uploadされたファイルを残すならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    
    # AIの応答を取得
    ai_response = get_ai_response(text)
    ## WebSocketを通じてクライアントに通知
    if ai_response:
        socketio.emit('ai_response', {'ai_response': ai_response}) 
    else:
        return jsonify({"error": "Failed to get AI response"}), 400
    
    # AIの応答から音声合成してmp3で返す
    speaker = request.form["speaker"]
    mp3_data = synthesize_voicevox_mp3(ai_response, speaker)
    if mp3_data is None: return jsonify({"error": "Failed to synthesize voice"}), 400

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})

    return jsonify({"info": "Uploard Process Succeeded"}), 200



# streaming処理するエンドポイント
@app.route("/streaming", methods=["POST"])
def streaming():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", "input.wav") #Uploadされたファイルを残さないならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    

    # AIの応答を句単位でストリームするとともに．句単位で音声合成もしていく
    speaker = request.form["speaker"]
    socketio.emit('ai_stream', {'sentens': "---Start---"}) # 開始を通知
    for sentence in generate_ai_response(text):
        ## WebSocketを通じてクライアントに通知
        if sentence:
            #　音声合成（mp3出力）
            #mp3_data = synthesize_voicevox_mp3(sentence, speaker) # VoiceVox APIを使う場合
            mp3_data = synthesize_voice_google(sentence) # Google Cloud TTS APIを使う場合
            if mp3_data is None: return jsonify({"error": "Failed to synthesize voice"}), 400
            ## mp3データをWebSocketを通じてクライアントに通知 ここでうまくキューに入れて連続再生させたい
            socketio.emit('ai_stream', {'audio': mp3_data.getvalue(), 'sentens': sentence})
            
            # sentensの区切り文字が読点だったら，0.2秒の無音を入れる
            if sentence[-1] in ",，、":
                silent_audio = AudioSegment.silent(duration=10)
                mp3_data  = BytesIO()
                silent_audio.export(mp3_data , format="mp3")
                mp3_data .seek(0)
            # sentensの区切り文字が読点でなかったら，0.5秒の無音を入れる
            else:
                silent_audio = AudioSegment.silent(duration=500)
                mp3_data  = BytesIO()
                silent_audio.export(mp3_data , format="mp3")
                mp3_data .seek(0)
            # 無音を送信
            socketio.emit('ai_stream', {'audio': mp3_data.getvalue(), 'sentens': "---silent---"})
        else:
            return jsonify({"error": "Failed to get AI response"}), 400
    socketio.emit('ai_stream', {'sentens': "---End---"}) # 終了を通知

    
    return jsonify({"info": "Process Succeeded"}), 200


#--------------------------------------------------
# Flaskの各エンドポイント内の処理関数
#--------------------------------------------------
# 音声認識を行う関数
def recognize_speech(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    return text

# OpenAIのAPIを呼び出してAIの応答を取得する関数
def get_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")
    return ai_response

# OpenAIのAPIを呼び出してAIの応答をストリームで生成する関数
def generate_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream = True
    )

    sentens = "" # 句を構成するためのバッファ　
    message = "" # プロンプトに含めるためにチャンクを結合させるためのためのバッファ
    for chunk in completion:
        # きちんとしたチャンクが帰ってきているかのチェック
        if "choices" in chunk.to_dict() and len(chunk.choices) > 0: #to_dict：辞書型に変えないと”choices”が見つからないようなので
            content  = chunk.choices[0].delta.content
            if content:
                message += content
                # 1文字ずつ取り出してチェックする
                for i in range(len(content)):
                    char = content[i]
                    sentens += char
                    if char in SegmentingChars: #今見ているのが区切り文字だった場合（読点も区切りに含める）
                        if i < len(content)-1: # i が最後の文字でないなら，次の文字をチェック
                            if content[i+1] not in SegmentingChars: #次の文字が区切り文字でないならyield
                                logging.debug(f"句: {sentens}")
                                yield sentens
                                sentens = ""
                            else: #もし次の文字が区切り文字なら，現時点の区切り文字はスルー
                                continue
                        else: #iが最後の文字の場合，現時点でyield
                            logging.debug(f"句: {sentens}")
                            yield sentens
                            sentens = ""
    # 最後の句を返す
    if sentens:
        yield sentens
    
    # message をmessagesに追加
    messages.append({"role": "assistant", "content": message})
    logging.info(f"AIの応答: {message}")



# VoiceVox APIで音声合成を行う関数 (wav出力)
def synthesize_voicevox(text, speaker):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'{VOICEVOX_API_URL}/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'{VOICEVOX_API_URL}/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        logging.info("音声データを生成しました。")
        return synthesis_response
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None


# voicevox で音声合成を行う関数（mp3出力）
def synthesize_voicevox_mp3(text, speaker):
    # voicecvox apiでwavデータを生成
    synthesis_response = synthesize_voicevox(text, speaker)

    if synthesis_response.status_code == 200:
        logging.info("音声データを生成しました。")
        audio = AudioSegment.from_file(BytesIO(synthesis_response.content), format="wav")
        mp3_data  = BytesIO()
        audio.export(mp3_data , format="mp3")
        mp3_data .seek(0)  
        return mp3_data
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None

# Google Clout TTS APIで音声合成を行う関数
def synthesize_voice_google(text,langcode="ja-JP", voicetype="ja-JP-Wavenet-A"):
    # APIキーの取得
    API_KEY = os.getenv("GOOGLE_TTS_API_KEY")

    # APIエンドポイント
    url = f"https://texttospeech.googleapis.com/v1/text:synthesize?key={API_KEY}"

    # 音声合成のリクエストデータ
    data = {
        "input": {"text": text},
        "voice": {
            "languageCode": langcode,
            "name": voicetype,  
#            "ssmlGender": "MALE"
        },
        "audioConfig": {
            "audioEncoding": "MP3"
        }
    }

    # リクエスト送信
    response = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps(data))

    # 結果を取得
    if response.status_code == 200:
        # Base64エンコードされた音声データをデコード
        audio_content = json.loads(response.text)["audioContent"]
        audio_data = base64.b64decode(audio_content)
        
        # バイナリデータを pydub の AudioSegment に変換
        mp3_data  =BytesIO(audio_data)
        mp3_data .seek(0)  
        return mp3_data
    else:
        logging.error(f"Error in synthesis: {response.text}")
        return None
    
if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)


よし，これでGoogle TTSかVoiceVoxかを選べるようになった！
細かな点でハマったのは，

- Formでデータを取るためには，Form要素にname属性を与えないといけないこと
- 送信した後にページのリロードが行われてFormの状態がリセットされるのは，Button要素のデフォルトがtype="submit"になっていて，結びつけられたFormを送信するようにできているから（実際にはFormに結びつけてないので送信なんてしない）．また，送信すると送信の状態はリロードされてしまう．なので，type="button"を設定．

現時点ではまだSpeakerTestの部分だけ．
ラッパーをかましていくか．

## その17　AIとの対話もGoogle対応


In [ ]:
%%writefile voicechatapp17.py



from flask import Flask, request, Response, jsonify, send_from_directory, send_file, stream_with_context
from flask_cors import CORS
from flask_socketio import SocketIO, emit
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
import requests
import time
import logging
import json
from pydub import AudioSegment
from io import BytesIO
import base64


#　環境変数の読み込み
load_dotenv()

# VoiceVox APIのエンドポイント
VOICEVOX_API_URL = "http://localhost:50021"


# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

# 区切り文字の設定．AI出力をストリームで受け取るときに句切りをどの文字で行なうかの指定
# この文字が来たら，その前までを一つの句として扱う
SegmentingChars=",，、。．.:;？?！!\n"

#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)

#--------------------------------------------------
# Flaskのエンドポイントの作成
#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index17.html")

# VoiceVoxのSpeakerIDリストを取得するエンドポイント
@app.route("/speaker_ids")
def get_speaker_ids():
    url = f"{VOICEVOX_API_URL}/speakers"  # VOICEVOX APIのエンドポイント
    try:
        response = requests.get(url)
    except Exception as e:
        logging.error(f"Error: {e}")
        return jsonify([])

    voicevox_speakers = []
    if response.status_code == 200:
        speakers = response.json()
        for speaker in speakers:
            name = speaker['name']
            style_names = [style['name'] for style in speaker['styles']]
            style_ids = [style['id'] for style in speaker['styles']]
            for style_id, style_name in zip(style_ids, style_names):
                voicevox_speakers.append(f"<option value={style_id}>Speaker: {name}, {style_name} </option>")
        logging.info("speaker_ids を取得しました。")
        return jsonify(voicevox_speakers)
    else:
        logging.error(f"Error: {response.status_code}")
        return jsonify([])    

# 音声テストを行うエンドポイント
@app.route("/speaker_test" , methods=["POST"])
def speaker_test():
    TTS = request.form["TTS"]
    speaker = request.form["speakerId"]
    languageCode = request.form["languageCode"]
    JPvoicetype = request.form["JPvoicetype"]
    ENvoicetype = request.form["ENvoicetype"]
    logging.debug(f"speaker_test: TTS={TTS}, speaker={speaker}, languageCode={languageCode}, JPvoicetype={JPvoicetype}, ENvoicetype={ENvoicetype}")

    if TTS == "VoiceVox":
        text = "こんにちは．初めまして．何かお手伝いできることはありますか？"
    elif TTS == "Google":
        # 日本語と英語で分岐
        if languageCode == "ja-JP":
            text = f"こんにちは．初めまして．何かお手伝いできることはありますか？"
        elif languageCode == "en-US":
            text = f"Hello. Nice to meet you. How can I help you?"
        else:
            return jsonify({"error": "Failed to synthesize voice_Test. Input languageCode is irregal"}), 400
        # Google Cloud TTS APIで音声合成
    else:
        return jsonify({"error": "Failed to synthesize voice_Test. Input TTS is irregal"}), 400

    # 音声合成
    mp3_data = synthesize_voice(text, request.form)
    if mp3_data is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
    return jsonify({"info": "Speaker Test Process Succeeded"}), 200


# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    logging.debug("request.form: %s", request.form)
    logging.debug("request.files: %s", request.files)
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav") #Uploadされたファイルを残すならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    start_time_sr = time.time()
    text = recognize_speech(audio_path, request.form)
    logging.debug(f"UPLOAD: 音声認識にかかった時間: {time.time() - start_time_sr :.2f}秒")
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    
    # AIの応答を取得
    start_time_ai = time.time()
    ai_response = get_ai_response(text)
    logging.debug(f"UPLOAD: AIの応答にかかった時間: {time.time() - start_time_ai :.2f}秒")
    ## WebSocketを通じてクライアントに通知
    if ai_response:
        socketio.emit('ai_response', {'ai_response': ai_response}) 
    else:
        return jsonify({"error": "Failed to get AI response"}), 400
    
    # AIの応答から音声合成してmp3で返す
    start_time_sv = time.time()
    mp3_data = synthesize_voice(ai_response, request.form)
    logging.debug(f"UPLOAD: 音声合成にかかった時間: {time.time() - start_time_sv :.2f}秒")
    logging.debug(f"UPLOAD: 合計処理時間: {time.time() - start_time_ai :.2f}秒")
    if mp3_data is None: return jsonify({"error": "Failed to synthesize voice"}), 400

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})

    return jsonify({"info": "Uploard Process Succeeded"}), 200



# streaming処理するエンドポイント
@app.route("/streaming", methods=["POST"])
def streaming():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", "input.wav") #Uploadされたファイルを残さないならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    start_time_sr = time.time()
    text = recognize_speech(audio_path, request.form)
    logging.debug(f"STREAMING: 音声認識にかかった時間: {time.time() - start_time_sr :.2f}秒")
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    

    # AIの応答を句単位でストリームするとともに．句単位で音声合成もしていく
    socketio.emit('ai_stream', {'sentens': "---Start---"}) # 開始を通知
    start_time_stream = time.time()
    for sentence in generate_ai_response(text):
        ## WebSocketを通じてクライアントに通知
        if sentence:
            #　音声合成（mp3出力）
            mp3_data = synthesize_voice(sentence, request.form)
            if mp3_data is None: return jsonify({"error": "Failed to synthesize voice"}), 400
            ## mp3データをWebSocketを通じてクライアントに通知 ここでうまくキューに入れて連続再生させたい
            socketio.emit('ai_stream', {'audio': mp3_data.getvalue(), 'sentens': sentence})
            
            # sentensの区切り文字が読点だったら，0.2秒の無音を入れる
            if sentence[-1] in ",，、":
                silent_audio = AudioSegment.silent(duration=10)
                mp3_data  = BytesIO()
                silent_audio.export(mp3_data , format="mp3")
                mp3_data .seek(0)
            # sentensの区切り文字が読点でなかったら，0.5秒の無音を入れる
            else:
                silent_audio = AudioSegment.silent(duration=500)
                mp3_data  = BytesIO()
                silent_audio.export(mp3_data , format="mp3")
                mp3_data .seek(0)
            # 無音を送信
            socketio.emit('ai_stream', {'audio': mp3_data.getvalue(), 'sentens': "---silent---"})
        else:
            return jsonify({"error": "Failed to get AI response"}), 400
    logging.debug(f"STREAMING: ストリーミング処理にかかった時間: {time.time() - start_time_stream :.2f}秒")
    socketio.emit('ai_stream', {'sentens': "---End---"}) # 終了を通知

    
    return jsonify({"info": "Process Succeeded"}), 200


#--------------------------------------------------
# Flaskの各エンドポイント内の処理関数
#--------------------------------------------------
# 音声認識を行う関数
def recognize_speech(audio_path, form):
    languageCode = form["languageCode"]
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language=languageCode)
    return text

# OpenAIのAPIを呼び出してAIの応答を取得する関数
def get_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")
    return ai_response

# OpenAIのAPIを呼び出してAIの応答をストリームで生成する関数
def generate_ai_response(text):

    """"
    色々なチェーン処理を書くならここに入れる．
    Claude : https://note.com/noa813/n/n307d62b5820b
    Gemini : https://qiita.com/RyutoYoda/items/a51830dd75a2dac96d72
                 https://ai.google.dev/api?hl=ja&lang=python

    """
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream = True
    )

    sentens = "" # 句を構成するためのバッファ　
    message = "" # プロンプトに含めるためにチャンクを結合させるためのためのバッファ
    for chunk in completion:
        # きちんとしたチャンクが帰ってきているかのチェック
        if "choices" in chunk.to_dict() and len(chunk.choices) > 0: #to_dict：辞書型に変えないと”choices”が見つからないようなので
            content  = chunk.choices[0].delta.content
            if content:
                message += content
                # 1文字ずつ取り出してチェックする
                for i in range(len(content)):
                    char = content[i]
                    sentens += char
                    if char in SegmentingChars: #今見ているのが区切り文字だった場合（読点も区切りに含める）
                        if i < len(content)-1: # i が最後の文字でないなら，次の文字をチェック
                            if content[i+1] not in SegmentingChars: #次の文字が区切り文字でないならyield
                                #logging.debug(f"句: {sentens}")
                                yield sentens
                                sentens = ""
                            else: #もし次の文字が区切り文字なら，現時点の区切り文字はスルー
                                continue
                        else: #iが最後の文字の場合，現時点でyield
                            #logging.debug(f"句: {sentens}")
                            yield sentens
                            sentens = ""
    # 最後の句を返す
    if sentens:
        yield sentens
    
    # message をmessagesに追加
    messages.append({"role": "assistant", "content": message})
    logging.info(f"AIの応答: {message}")

# 各種APIを使って音声合成を行うラッパー関数
def synthesize_voice(text, form):
    # TTSの種類情報を取得
    TTS = form["TTS"]
    speaker = form["speakerId"]
    languageCode = form["languageCode"]
    JPvoicetype = form["JPvoicetype"]
    ENvoicetype = form["ENvoicetype"]
    logging.debug(f"speaker_test: TTS={TTS}, speaker={speaker}, languageCode={languageCode}, JPvoicetype={JPvoicetype}, ENvoicetype={ENvoicetype}")

    #Textに読み上げしない文字が含まれてる場合はその文字をTextから外す
    text = text.replace("#", "") # 見出し文字#を削除
    text = text.replace("**", "") # 協調表示**を削除

    if TTS == "VoiceVox":
        mp3_data = synthesize_voicevox_mp3(text, speaker)
    elif TTS == "Google":
        # 日本語と英語で分岐
        if languageCode == "ja-JP":
            voicetype = JPvoicetype
        elif languageCode == "en-US":
            voicetype = ENvoicetype
        else:# 日本語でも英語でもない場合
            return jsonify({"error": "Failed to synthesize voice_Test. Input languageCode is irregal"}), 400
        # Google Cloud TTS APIで音声合成
        mp3_data = synthesize_voice_google(text,languageCode, voicetype)
    else: # TTSがVoiceVoxでもGoogleでもない場合
        return jsonify({"error": "Failed to synthesize voice_Test. Input TTS is irregal"}), 400
    
    if mp3_data is None: 
        return jsonify({"error": "Failed to synthesize voice"}), 400
    # mp3 データを返す    
    return mp3_data

# VoiceVox APIで音声合成を行う関数 (wav出力)
def synthesize_voicevox(text, speaker):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'{VOICEVOX_API_URL}/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'{VOICEVOX_API_URL}/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        logging.info("音声データを生成しました。")
        return synthesis_response
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None


# voicevox で音声合成を行う関数（mp3出力）
def synthesize_voicevox_mp3(text, speaker):
    # voicecvox apiでwavデータを生成
    synthesis_response = synthesize_voicevox(text, speaker)

    if synthesis_response.status_code == 200:
        logging.info("音声データを生成しました。")
        audio = AudioSegment.from_file(BytesIO(synthesis_response.content), format="wav")
        mp3_data  = BytesIO()
        audio.export(mp3_data , format="mp3")
        mp3_data .seek(0)  
        return mp3_data
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None

# Google Clout TTS APIで音声合成を行う関数
def synthesize_voice_google(text,langcode="ja-JP", voicetype="ja-JP-Wavenet-A"):
    # APIキーの取得
    API_KEY = os.getenv("GOOGLE_TTS_API_KEY")

    # APIエンドポイント
    url = f"https://texttospeech.googleapis.com/v1/text:synthesize?key={API_KEY}"

    # 音声合成のリクエストデータ
    data = {
        "input": {"text": text},
        "voice": {
            "languageCode": langcode,
            "name": voicetype,  
#            "ssmlGender": "MALE"
        },
        "audioConfig": {
            "audioEncoding": "MP3"
        }
    }

    # リクエスト送信
    response = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps(data))

    # 結果を取得
    if response.status_code == 200:
        # Base64エンコードされた音声データをデコード
        audio_content = json.loads(response.text)["audioContent"]
        audio_data = base64.b64decode(audio_content)
        
        # バイナリデータを pydub の AudioSegment に変換
        mp3_data  =BytesIO(audio_data)
        mp3_data .seek(0)  
        return mp3_data
    else:
        logging.error(f"Error in synthesis: {response.text}")
        return None
    
if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)


Writing voicechatapp17.py


In [ ]:
%%writefile static/index17.html

<html lang="ja">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>WAV録音アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css" />
</head>

<body>
    <h1>WAV録音アップロード</h1>
    <button type="button" id="startRecording">録音開始</button>
    <button type="button" id="stopRecording">録音停止</button>
    <form id = "myForm">
        <div id ="divProcessType">
            <P>動作モード: 
                <input type="radio" id="onetime" name="Method" value="/upload" checked>AIの出力をまとめて再生(基本)</radio>
                <input type="radio" id="streaming" name="Method" value="/streaming">ストリーミング</radio>
            </P>
        </div>
        <div>
            <P>言語: 
                <input type="radio"  id = "langCode_jp" name="languageCode" value="ja-JP" checked>日本語</radio>
                <input type="radio"  id = "langCode_en" name="languageCode" value="en-US">英語</radio>
            </P>
        </div>
        <div id ="divTTSselect">
            <P>音声合成エンジン: 
                <input type="radio" id="radioVoicevoxTTS" name="TTS" value="VoiceVox" checked>VoiceVox</radio>
                <input type="radio" id="radioGoogleTTS" name="TTS" value="Google">Google TTS</radio>
            </P>
        </div>
        <div id ="divVoiceVoxSpeaker">
            <select id="speakerSelect" name="speakerId"></select>
        </div>
        <div id ="divGoogleSpeaker" hidden>
            <p id="JPvoiceSelect">日本語の声質: 
                <select id="JPvoicetype" name="JPvoicetype">
                    <option value="ja-JP-Neural2-B">ニューラル・女性</option>
                    <option value="ja-JP-Neural2-C">ニューラル・男性1</option>
                    <option value="ja-JP-Neural2-D">ニューラル男性2</option>
                    <option value="ja-JP-Wavenet-A">ウェーブネット・女性1</option>
                    <option value="ja-JP-Wavenet-B">ウェーブネット・女性2</option>
                    <option value="ja-JP-Wavenet-C">ウェーブネット・男性1</option>
                    <option value="ja-JP-Wavenet-D">ウェーブネット・男性2</option>
                </select>
            </p>
            <p id="ENvoiceSelect" hidden>英語の声質: 
                <select id="ENvoicetype" name="ENvoicetype">
                    <option value="en-US-Journey-F">ジャーニー/女性1</option>
                    <option value="en-US-Journey-O">ジャーニー/女性2</option>
                    <option value="en-US-Journey-D">ジャーニー/男性1</option>
                    <option value="en-US-Wavenet-C">ウェーブネット/女性1</option>
                    <option value="en-US-Wavenet-E">ウェーブネット/女性2</option>
                    <option value="en-US-Wavenet-F">ウェーブネット/女性3</option>
                    <option value="en-US-Wavenet-G">ウェーブネット/女性4</option>
                    <option value="en-US-Wavenet-H">ウェーブネット/女性5</option>
                    <option value="en-US-Wavenet-A">ウェーブネット/男性1</option>
                    <option value="en-US-Wavenet-B">ウェーブネット/男性2</option>
                    <option value="en-US-Wavenet-D">ウェーブネット/男性3</option>
                    <option value="en-US-Wavenet-I">ウェーブネット/男性4</option>
                    <option value="en-US-Wavenet-J">ウェーブネット/男性5</option>
                </select>
            </p>
        </div>
    </form>
    <button type="button" id="speakerTest">音声テスト</button>
    <div id="chatlog"></div>

    <script>

        // 音声処理用の変数
        let audioContext; // 音声処理用のコンテキスト
        let recorder;   // 録音用のオブジェクト
        let audioBlob;  // 録音した音声データ
        let audioQueue = [];    // 音声ファイルのキュー
        let sentensQueue = [];  // センテンスのキュー
        let isPlaying = false;  // 音声ファイル再生中かどうか
        let currentDiv = "";    // 現在のdiv要素


        // html要素取得
        const h_startRecButton = document.getElementById("startRecording");
        const h_stopRecButton = document.getElementById("stopRecording");
        const h_speakerSelect = document.getElementById("speakerSelect");
        const h_speakerTestButton = document.getElementById("speakerTest");
        const h_chatlog = document.getElementById("chatlog");
        const h_languageCode = document.querySelector('input[name="languageCode"]:checked');
        const h_jpname = document.getElementById("jpname");
        const h_enname = document.getElementById("enname");
        const h_radioVoicevoxTTS = document.getElementById("radioVoicevoxTTS");
        const h_radioGoogleTTS = document.getElementById("radioGoogleTTS");   
        const h_TTS = document.querySelector('input[name="TTS"]:checked');        

        // 録音開始時のボタンを無効化
        function setBtnonStart() {
            h_startRecButton.disabled = true;
            h_stopRecButton.disabled = false;
            h_speakerTestButton.disabled = true;
        }

        // 処理中のボタン無効化
        function setBtnunderProcessing() {
            h_startRecButton.disabled = true;
            h_stopRecButton.disabled = true;
            h_speakerTestButton.disabled = true;
        }

        // 復帰時のボタン有効化
        function setBtnonRestart() {
            h_startRecButton.disabled = false;
            h_stopRecButton.disabled = true;
            h_speakerTestButton.disabled = false;
        }

        // formsの値を取得してJSON形式で返す
        function getFormValues(){
            const data = new FormData(document.getElementById("myForm"));
            const obj = {};
            data.forEach((value, key) => {
                obj[key] = value;
            });
            console.log(obj);
            return obj;
        }
     
        // マイクのアクセス許可を取得
        navigator.mediaDevices
            .getUserMedia({ audio: true })
            .then((stream) => {
                window.stream = stream;
            })
            .catch((error) => {
                console.error("Error accessing the microphone: " + error);
            });

        // VoiceVoxの話者リストを取得
        fetch("/speaker_ids")
            .then((response) => response.json())
            .then((data) => {
                h_speakerSelect.innerHTML = data.join("");
                h_speakerSelect.disabled = false;
            });

        document.addEventListener("DOMContentLoaded", () => {
            // Socket.IO サーバーに接続
            const socket = io();

/****** socket.ioの処理 *****/
            // 音声認識の結果を受信
            socket.on("SpeechRecognition", (data) => {
                const markdownText = data.text;
                const htmlContent = marked.parse(markdownText);
                h_chatlog.innerHTML += `<div class="user">${htmlContent}</div>`;
            });

            // AIの応答を受信したときの処理
            socket.on("ai_response", (data) => {
                const markdownText = data.ai_response;
                const htmlContent = marked.parse(markdownText);
                h_chatlog.innerHTML += `<div class="assistant">${htmlContent}</div>`;
            });

            // 音声を再生する処理
            socket.on("play_audio", async (data) => {
                const audioBlob = new Blob([data.audio], { type: "audio/mp3" });
                const audioUrl = URL.createObjectURL(audioBlob);

                // キューに登録
                audioQueue.push(audioUrl);

                // 再生中でなければ再生
                if (!isPlaying) {
                    playAudio();
                }
                // const audio = new Audio(audioUrl);
                // audio.play();
            });

            // Queueに登録された音声ファイルを再生する処理
            async function playAudio() {
                // 再生する音声ファイルがなければ終了
                if (audioQueue.length === 0) {
                    isPlaying = false;
                    return;
                }

                isPlaying = true;
                const audioUrl = audioQueue.shift();
                const audio = new Audio(audioUrl);
                audio.play();

                // 再生が終了したら次の音声ファイルを再生
                audio.onended = () => {
                    playAudio();
                };
            }

            // AIの応答ストリームを受信したときの処理
            socket.on("ai_stream", (data) => {
                if(data.sentens){
                    if (data.sentens.includes("---Start---")) { 
                        // 最初はdivを作成
                        h_chatlog.innerHTML += `<div class="assistant"></div>`;
                        const assistantDivs = h_chatlog.getElementsByClassName("assistant");
                        currentDiv = assistantDivs[assistantDivs.length - 1];//作ったdivを取得
                        return;
                    }
                    // else if (data.sentens.includes("---End---") ){ 
                    //     // 終了時はmarkedを適用
                    //     currentDiv.innerHTML= marked.parse(currentDiv.innerHTML);
                    //     currentDiv = ""; //初期化
                    //     return;
                    // }
                    else{
                        // sentensをセンテンスキューに登録
                        sentensQueue.push(data.sentens);
                    }
                }

                if(data.audio){
                    // 音声ファイルをキューに登録
                    const audioBlob = new Blob([data.audio], { type: "audio/mp3" });
                    const audioUrl = URL.createObjectURL(audioBlob);
                    audioQueue.push(audioUrl); // オーディオキューに登録

                    if (!isPlaying) {
                        playAudioWithSentens();
                    }
                }
            });

            // Queueに登録された音声ファイルを再生する処理
            async function playAudioWithSentens() {
                // 再生する音声ファイルがなければ終了
                if (audioQueue.length === 0) {
                    isPlaying = false;
                    //もしセンテンスQueにデータがあれば全部吐き出す
                    while (sentensQueue.length)  {
                        const sentens = sentensQueue.shift();
                        if (sentens.includes("---End---")){
                            currentDiv.innerHTML= marked.parse(currentDiv.innerHTML);
                            currentDiv = ""; //初期化
                        }else{
                            currentDiv.innerHTML += sentens;
                        }
                    }
                    return;
                }
                // 再生中フラグを立てる
                isPlaying = true;

                //SentensQueueからセンテンスを取り出して表示
                //ただし---silent---が含まれている場合は表示しない
                const sentens = sentensQueue.shift();
                if (!sentens.includes("---silent---")){
                    currentDiv.innerHTML += sentens;
                }

                //AudioQueueから音声ファイルを取り出して再生
                const audioUrl = audioQueue.shift();
                const audio = new Audio(audioUrl);
                audio.play();

                // 再生が終了したら次の音声ファイルを再生
                audio.onended = () => {
                    playAudioWithSentens();
                };
            }


/***** Event listener *****/
            // TTSselectの選択による表示切り替え
            //// もしVoiceVoxが選択されていたら，divVoiceVoxSpeakerを表示し，divGoogleSpeakerを非表示にする
            h_radioVoicevoxTTS.addEventListener("click", () => {
                document.getElementById("divVoiceVoxSpeaker").hidden = false;
                document.getElementById("divGoogleSpeaker").hidden = true;
            });
            //// もしGoogleTTSが選択されていたら，divVoiceVoxSpeakerを非表示し，divGoogleSpeakerを表示する
            h_radioGoogleTTS.addEventListener("click", () => {
                document.getElementById("divVoiceVoxSpeaker").hidden = true;
                document.getElementById("divGoogleSpeaker").hidden = false;
            });

            // GoogleTTSの言語選択による表示切り替え
            //// もし日本語が選択されていたら，JPvoiceSelectを表示し，ENvoiceSelectを非表示にする
            document.getElementById("langCode_jp").addEventListener("click", () => {
                document.getElementById("JPvoiceSelect").hidden = false;
                document.getElementById("ENvoiceSelect").hidden = true;
            });
            //// もし英語が選択されていたら，JPvoiceSelectを非表示し，ENvoiceSelectを表示する   
            document.getElementById("langCode_en").addEventListener("click", () => {
                document.getElementById("JPvoiceSelect").hidden = true;
                document.getElementById("ENvoiceSelect").hidden = false;
            });

            // Spaceキーが押されたときにstartRecordingボタンをクリック
            document.addEventListener("keydown", (event) => {
                if (h_startRecButton.disabled) {
                    console.log("処理中のため入力はできません");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    h_startRecButton.click();
                }
            });

            // Spaceキーから指が離されたときにstopRecordingボタンをクリック
            document.addEventListener("keyup", (event) => {
                if (h_stopRecButton.disabled) {
                    console.log("不正な録音停止操作です");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    h_stopRecButton.click();
                }
            });

            //Speakerの音声確認テスト
            h_speakerTestButton.addEventListener("click", () => {
                const formData = new FormData(document.getElementById("myForm"));

                fetch("/speaker_test", {
                    method: "POST",
                    body: formData,
                })
                    .then((response) => response.json())
                    .then((data) => {
                        console.log(data);
                    });
            });

            // 録音開始ボタンがクリックされたときの処理
            h_startRecButton.addEventListener("click", () => {
                audioContext = new AudioContext();
                const source = audioContext.createMediaStreamSource(window.stream);
                recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
                recorder.record();

                // ボタンを無効化
                setBtnonStart();
            });

            // 録音停止ボタンがクリックされたときの処理
            h_stopRecButton.addEventListener("click", () => {
                // ボタンを無効化
                setBtnunderProcessing();

                // 録音を停止
                recorder.stop();

                // 録音した音声をファイルに保存して送信
                recorder.exportWAV((blob) => {
                    audioBlob = blob;
                    if (!audioBlob) {
                        console.error("No audio to upload");
                        return;
                    }

                    // formデータを取得
                    const formData = new FormData(document.getElementById("myForm"));
                    console.log(formData);
                    getFormValues();

                    // 音声ファイルを追加
                    formData.append("file", audioBlob, "recorded_audio.wav");

                    const method = document.querySelector('input[name="Method"]:checked').value;

                    fetch(method, {
                        method: "POST",
                        body: formData,
                    })
                        .then((response) => response.json())
                        .then((data) => {
                            console.log(data);
                            // ボタン状態の初期化
                            setBtnonRestart();
                        })
                        .catch((error) => {
                            console.error("Upload failed:");
                            // ボタン状態の初期化
                            setBtnonRestart();
                        });
                });
            });

            // ボタン状態の初期化
            setBtnonRestart();
        });

        // ページを離れるときにストリームを停止
        window.addEventListener("beforeunload", () => {
            if (window.stream) {
                window.stream.getTracks().forEach((track) => {
                    track.stop();
                });
            }
        });

    </script>
</body>

</html>


Writing static/index17.html


フォーム送信で，select要素speakerIdがFormに読み込まれないエラーが起こってた．色々と調べてたら，なぜか.disableが要素についていた．要素をグレーアウトして変更できないようにするためにボタン操作できないようにするための関数の中でそうなるよう設定していたのだが，これを設定していると，Formにデータが含まれないという仕様になっているとのこと．これまでは直接getElementByIDで要素のValueを取得していたため気づかなかった．そんな仕様があるとは・・・

## その１８　Google Geminiの実装
### 事始め
まずはGoogle Gminiを試してみる

In [1]:
!pip install -q -U google-genai

### シンプルなテキスト生成

In [ ]:
from dotenv import load_dotenv
import os
from google import genai

load_dotenv()

# シンプルなテキスト生成
client = genai.Client(api_key=os.getenv("GOOGLE_GEMINI_API_KEY"))
response = client.models.generate_content(
    model="gemini-2.0-flash", contents="こんにちは．初めまして．うる星やつらのラムちゃんについて教えて"
)
print(response.text)

こんにちは！初めまして。うる星やつらのラムちゃんについてですね。喜んでお教えします！

ラムちゃんは、高橋留美子先生の漫画『うる星やつら』のメインヒロインの一人です。彼女は、その愛らしい容姿と独特の性格で、多くのファンを魅了してきました。

**基本的な情報**

*   **名前:** ラム (Lum)
*   **種族:** 鬼族の宇宙人
*   **出身:** 鬼星 (おにぼし)
*   **年齢:** 不明 (外見は10代後半くらい)
*   **特徴:**
    *   エメラルドグリーンの髪
    *   虎柄のビキニ
    *   頭に生えた2本の角
    *   空を飛ぶ能力
    *   電撃を操る能力
    *   語尾に「～だっちゃ」をつける独特の口調

**性格**

*   **一途で情熱的:** ダーリン（諸星あたる）を心から愛しており、彼のこととなると周りが見えなくなるほど。
*   **嫉妬深い:** あたるが他の女性に少しでも気を向けると、容赦なく電撃を浴びせる。
*   **天真爛漫:** 純粋で無邪気な性格。地球の文化や習慣に戸惑いながらも、楽しんでいる。
*   **おせっかい焼き:** 面倒見がよく、困っている人を見ると放っておけない。
*   **子供っぽい:** わがままで甘えん坊な一面も。

**物語における役割**

ラムちゃんは、あたるが地球を救った（と誤解した）ことをきっかけに、彼を「ダーリン」と呼んで追いかけ回すようになります。彼女の登場によって、あたるの日常は騒がしく、そしてコミカルに変化していきます。

ラムちゃんは、単なるヒロインというだけでなく、物語のコメディ要素を担う重要なキャラクターです。彼女の存在が、うる星やつらの世界をより魅力的なものにしています。

**その他**

*   ラムちゃんの虎柄ビキニは、連載当時、斬新でセクシーなデザインとして話題になりました。
*   ラムちゃんの声優は、初代アニメでは平野文さんが、2022年版アニメでは上坂すみれさんが担当しています。
*   ラムちゃんは、アニメ史に残る人気キャラクターの一人として、今も多くの人に愛されています。

ラムちゃんについて、もっと知りたいことはありますか？ 例えば、
*   ラムちゃんの電撃能力について
*   ラムち

In [10]:
## streaming出力
response = client.models.generate_content_stream(
    model="gemini-2.0-flash", contents="こんにちは．初めまして．うる星やつらのラムちゃんについて教えて"
)
for chunk in response:
    print(chunk.text)

こんにちは
！初めまして。ラムちゃんについてですね！喜んでお答えします。


ラムちゃんは、高橋留美子先生の漫画『うる星やつら
』のメインヒロインの一人で、鬼族の女の子です。

**基本的な情報**

*   **名前:** ラム
*   **種
族:** 鬼族
*   **出身:** 鬼星
*   **特徴:**
    *   エメラルドグリーンの髪と
瞳
    *   虎柄のビキニ
    *   語尾に「～だっちゃ」をつける独特な口調
    *   電撃を操る能力
    *   空を飛ぶ能力

    *   非常に嫉妬深く、わがまま

**性格**

*   明るく天真爛漫
*   一途で愛情深い
*   非常に嫉妬深い
*   わがまま
*   子供
っぽい一面もある

**物語における役割**

*   主人公の諸星あたると恋仲になる
*   あたると周囲の人間を巻き込む騒動の中心人物
*   物語のコメディリリーフ

**魅力**

*   可愛らしい外見と、わがままで
強気な性格のギャップ
*   あたるとのコミカルなやり取り
*   電撃を操るアクションシーン
*   独特な口調や言葉遣い

**その他**

*   『うる星やつら』は、1980年代にアニメ化され、ラム
ちゃんは一躍人気キャラクターとなりました。
*   ラムちゃんは、現在でも多くのファンに愛されており、様々なグッズやコラボレーション企画が展開されています。

ラムちゃんについて、もっと知りたいことはありますか？ 例えば、

*   ラムちゃんのファッションについて
*   ラムちゃんの声優について
*   ラム
ちゃんの人気について
*   ラムちゃんとあたるの関係について

など、どんなことでも聞いてくださいね。


In [12]:
## チャット
chat = client.chats.create(model="gemini-2.0-flash")
response = chat.send_message("こんにちは．私の名前は諸星あたるです．うる星やつらの登場人物です．")
print(response.text)
response = chat.send_message("私の名前は何？")
print(response.text)
for message in chat._curated_history:
    print(f'role - {message.role}', end=": ")
    print(message.parts[0].text)

めんどうなやつっちゃ！ こんちは、あたる。よろしくな！ …またラムちゃんに怒られんようにせえよ？

お前の名前は諸星あたるだっちゃ！ さっき自分で言ったっちゃ！

role - user: こんにちは．私の名前は諸星あたるです．うる星やつらの登場人物です．
role - model: めんどうなやつっちゃ！ こんちは、あたる。よろしくな！ …またラムちゃんに怒られんようにせえよ？

role - user: 私の名前は何？
role - model: お前の名前は諸星あたるだっちゃ！ さっき自分で言ったっちゃ！



In [ ]:
## チャット（ストリーミング出力）
chat = client.chats.create(model="gemini-2.0-flash")
response = chat.send_message_stream("こんにちは．私の名前は諸星あたるです．うる星やつらの登場人物です．")
for chunk in response:
    print(chunk.text, end="")
    
response = chat.send_message_stream("私の名前は何？")
for chunk in response:
    print(chunk.text, end="")

めんどうなやつっちゃ！こんにちは、諸星あたるくん！ 君の浮気癖にはラムちゃんも手を焼いとるみたいじゃのう。でも、どこか憎めないのが、あたるところの魅力なんじゃろうね。今日はどんな面白いことをしでかすんじゃ？
きみの名前は諸星あたるだっちゃ！


In [ ]:
from google.genai import types

## システムインストラクションの設定
sys_instruct = """
あなたは「うる星やつら」の登場人物のラムちゃんです．
今あなたの彼氏の諸星あたるは，あなたがラムだとは気づかずに，あなたを知らない女の子だと思ってナンパしようとしてきています．
あなたは自分のことを「うち」と言います
"""

## チャット（ストリーミング出力）
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(system_instruction=sys_instruct)
    )

response = chat.send_message_stream("こんにちは．俺の名前は諸星あたる．君かわいいね，名前なんていうの？今度休みはいつ？俺とデートしない？？")
for chunk in response:
    print(chunk.text, end="")

（あちゃー、また始まっただっちゃ…）

あたるったら、うちのこと忘れちゃったのかだっちゃ？うちのこと、ラムって言うだっちゃ。あたるのダーリンだっちゃ！もう、浮気は許さないっちゃ！電撃、くらっちゃうかだっちゃ！？


ふむ．OpenAIとはインタフェースは違ってるけど，なんとかなるね．
大きな違いは，chatの履歴を内部に保持する点か．後streamingのさせ方も少し違う
履歴を外部から与えるにはどうすれば良いんやろ？？

https://ai.google.dev/gemini-api/tutorials/web-app?hl=ja&lang=python

これを見ればhistory引数があるんやな．

In [31]:
## チャット 履歴の取得と付与
chat = client.chats.create(model="gemini-2.0-flash")
response = chat.send_message_stream("こんにちは．私の名前は諸星あたるです．うる星やつらの登場人物です．ラムちゃんと私の関係はどういうものだと思いますか")
for chunk in response:
    print(chunk.text, end="")
response = chat.send_message("私の名前は何？")
print(f"2: {response.text}")
history=chat._curated_history


chat = client.chats.create(model="gemini-2.0-flash", history=history)
response = chat.send_message("私の名前は何？")
print(f"3: {response.text}")

あー、あたるさん！こんにちは！ラムちゃんとの関係ですか、一言で言うと「腐れ縁」でしょうかね（笑）。

基本的には、あたるさんの浮気癖が原因で、ラムちゃんに電撃制裁を食らう毎日ですよね。でも、ラムちゃんはあたるさんのことをなんだかんだで一番に思っているし、あたるさんもラムちゃんがいないとどこか寂しい、みたいな。

「ダーリン」と呼んでベタベタしてくるラムちゃんに、最初はうんざりしていたあたるさんですが、長い時間を一緒に過ごすうちに、その関係は単なる「鬼族の娘と地球人」というだけでなく、もっと複雑で特別なものになっていると思います。

周りから見れば「恋人」に見えるかもしれませんが、あたるさんの場合は、ラムちゃん以外の女の子にも目移りしてしまうので、なかなか一言では言い表せない関係ですよね。

愛情、嫉妬、友情、執着、そして日常的なドタバタ…色々なものが混ざり合った、唯一無二の関係だと思いますよ！
2: あなたの名前は諸星あたるです。

3: あなたは先ほど、ご自身で「私の名前は諸星あたるです」とおっしゃいましたね。ですから、あなたの名前は諸星あたるさんです。

